In [25]:
import nba_py

In [26]:
from requests import get
from datetime import datetime
from nba_py.constants import League

HAS_PANDAS = True
try:
    from pandas import DataFrame
except ImportError:
    HAS_PANDAS = False

# Constants
TODAY = datetime.today()
BASE_URL = 'http://stats.nba.com/stats/{endpoint}'
HEADERS = {'user-agent': ('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5) '
                          'AppleWebKit/537.36 (KHTML, like Gecko) '
                          'Chrome/45.0.2454.101 Safari/537.36'),
           'referer': 'http://stats.nba.com/scores/'
          }


def _api_scrape(json_inp, ndx):
    """
    Internal method to streamline the getting of data from the json

    Args:
        json_inp (json): json input from our caller
        ndx (int): index where the data is located in the api

    Returns:
        If pandas is present:
            DataFrame (pandas.DataFrame): data set from ndx within the
            API's json
        else:
            A dictionary of both headers and values from the page
    """

    try:
        headers = json_inp['resultSets'][ndx]['headers']
        values = json_inp['resultSets'][ndx]['rowSet']
    except KeyError:
        # This is so ugly but this is what you get when your data comes out
        # in not a standard format
        try:
            headers = json_inp['resultSet'][ndx]['headers']
            values = json_inp['resultSet'][ndx]['rowSet']
        except KeyError:
            # Added for results that only include one set (ex. LeagueLeaders)
            headers = json_inp['resultSet']['headers']
            values = json_inp['resultSet']['rowSet']
    if HAS_PANDAS:
        return DataFrame(values, columns=headers)
    else:
        # Taken from www.github.com/bradleyfay/py-goldsberry
        return [dict(zip(headers, value)) for value in values]


def _get_json(endpoint, params):
    """
    Internal method to streamline our requests / json getting

    Args:
        endpoint (str): endpoint to be called from the API
        params (dict): parameters to be passed to the API

    Raises:
        HTTPError: if requests hits a status code != 200

    Returns:
        json (json): json object for selected API call
    """
    _get = get(BASE_URL.format(endpoint=endpoint), params=params,
               headers=HEADERS)
    # print _get.url
    _get.raise_for_status()
    return _get.json()


class Scoreboard:
    """ A scoreboard for all games for a given day
    Displays current games plus info for a given day

    Args:
        :month: Specified month (1-12)
        :day: Specified day (1-31)
        :year: Specified year (YYYY)
        :league_id: ID for the league to look in (Default is 00)
        :offset: Day offset from which to operate

    Attributes:
        :json: Contains the full json dump to play around with
    """
    _endpoint = 'scoreboard'

    def __init__(self,
                 month=TODAY.month,
                 day=TODAY.day,
                 year=TODAY.year,
                 league_id=League.NBA,
                 offset=0):
        self._game_date = '{month:02d}/{day:02d}/{year}'.format(month=month,
                                                                day=day,
                                                                year=year)
        self.json = _get_json(endpoint=self._endpoint,
                              params={'LeagueID': league_id,
                                      'GameDate': self._game_date,
                                      'DayOffset': offset})

    def game_header(self):
        return _api_scrape(self.json, 0)

    def line_score(self):
        return _api_scrape(self.json, 1)

    def series_standings(self):
        return _api_scrape(self.json, 2)

    def last_meeting(self):
        return _api_scrape(self.json, 3)

    def east_conf_standings_by_day(self):
        return _api_scrape(self.json, 4)

    def west_conf_standings_by_day(self):
        return _api_scrape(self.json, 5)

    def available(self):
        return _api_scrape(self.json, 6)

In [27]:
CURRENT_SEASON = '2015-16'

TEAMS = {
    'ATL': {
        'abbr': 'ATL',
        'city': 'Atlanta',
        'code': 'hawks',
        'conference': 'Eastern',
        'displayAbbr': 'ATL',
        'displayConference': 'Eastern',
        'division': 'Southeast',
        'id': '1610612737',
        'name': 'Hawks',
        'color': 'E2373E',
        'colors': ['E2373E', '002A5C', 'BAC4CA']
    }, 'BOS': {
        'abbr': 'BOS',
        'city': 'Boston',
        'code': 'celtics',
        'conference': 'Eastern',
        'displayAbbr': 'BOS',
        'displayConference': 'Eastern',
        'division': 'Atlantic',
        'id': '1610612738',
        'name': 'Celtics',
        'color': '007239',
        'colors': ['007239', 'AE8445', '982527', '000000']
    }, 'BKN': {
        'abbr': 'BKN',
        'city': 'Brooklyn',
        'code': 'nets',
        'conference': 'Eastern',
        'displayAbbr': 'BKN',
        'displayConference': 'Eastern',
        'division': 'Atlantic',
        'id': '1610612751',
        'name': 'Nets',
        'color': '000000',
        'colors': ['000000', 'FFFFFF']
    }, 'CHA': {
        'abbr': 'CHA',
        'city': 'Charlotte',
        'code': 'hornets',
        'conference': 'Eastern',
        'displayAbbr': 'CHA',
        'displayConference': 'Eastern',
        'division': 'Southeast',
        'id': '1610612766',
        'name': 'Hornets',
        'color': '00848E',
        'colors': ['00848E', '260F54', 'CCCCCC']
    }, 'CHI': {
        'abbr': 'CHI',
        'city': 'Chicago',
        'code': 'bulls',
        'conference': 'Eastern',
        'displayAbbr': 'CHI',
        'displayConference': 'Eastern',
        'division': 'Central',
        'id': '1610612741',
        'name': 'Bulls',
        'color': 'C60033',
        'colors': ['C60033', '000000']
    }, 'CLE': {
        'abbr': 'CLE',
        'city': 'Cleveland',
        'code': 'cavaliers',
        'conference': 'Eastern',
        'displayAbbr': 'CLE',
        'displayConference': 'Eastern',
        'division': 'Central',
        'id': '1610612739',
        'name': 'Cavaliers',
        'color': '860038',
        'colors': ['860038', '002D62', 'FDBA31']
    }, 'DAL': {
        'abbr': 'DAL',
        'city': 'Dallas',
        'code': 'mavericks',
        'conference': 'Western',
        'displayAbbr': 'DAL',
        'displayConference': 'Western',
        'division': 'Southwest',
        'id': '1610612742',
        'name': 'Mavericks',
        'color': '0063AF',
        'colors': ['0063AF', 'BAC4CA', '000000']
    }, 'DEN': {
        'abbr': 'DEN',
        'city': 'Denver',
        'code': 'nuggets',
        'conference': 'Western',
        'displayAbbr': 'DEN',
        'displayConference': 'Western',
        'division': 'Northwest',
        'id': '1610612743',
        'name': 'Nuggets',
        'color': '559FD6',
        'colors': ['559FD6', '006BB7', 'FEA927']
    }, 'DET': {
        'abbr': 'DET',
        'city': 'Detroit',
        'code': 'pistons',
        'conference': 'Eastern',
        'displayAbbr': 'DET',
        'displayConference': 'Eastern',
        'division': 'Central',
        'id': '1610612765',
        'name': 'Pistons',
        'color': 'EC003D',
        'colors': ['EC003D', '0058A6', '001D4A']
    }, 'GSW': {
        'abbr': 'GSW',
        'city': 'Golden State',
        'code': 'warriors',
        'conference': 'Western',
        'displayAbbr': 'GSW',
        'displayConference': 'Western',
        'division': 'Pacific',
        'id': '1610612744',
        'name': 'Warriors',
        'color': '0068B3',
        'colors': ['0068B3', 'FFC423']
    }, 'HOU': {
        'abbr': 'HOU',
        'city': 'Houston',
        'code': 'rockets',
        'conference': 'Western',
        'displayAbbr': 'HOU',
        'displayConference': 'Western',
        'division': 'Southwest',
        'id': '1610612745',
        'name': 'Rockets',
        'color': 'C60033',
        'colors': ['C60033', '000000']
    }, 'IND': {
        'abbr': 'IND',
        'city': 'Indiana',
        'code': 'pacers',
        'conference': 'Eastern',
        'displayAbbr': 'IND',
        'displayConference': 'Eastern',
        'division': 'Central',
        'id': '1610612754',
        'name': 'Pacers',
        'color': '001D4A',
        'colors': ['001D4A', 'FEAC2D', 'B0B2B5']
    }, 'LAC': {
        'abbr': 'LAC',
        'city': 'Los Angeles',
        'code': 'clippers',
        'conference': 'Western',
        'displayAbbr': 'LAC',
        'displayConference': 'Western',
        'division': 'Pacific',
        'id': '1610612746',
        'name': 'Clippers',
        'color': '00559A',
        'colors': ['00559A', 'EC003D']
    }, 'LAL': {
        'abbr': 'LAL',
        'city': 'Los Angeles',
        'code': 'lakers',
        'conference': 'Western',
        'displayAbbr': 'LAL',
        'displayConference': 'Western',
        'division': 'Pacific',
        'id': '1610612747',
        'name': 'Lakers',
        'color': 'FEA927',
        'colors': ['FEA927', '42186E', '000000']
    }, 'MEM': {
        'abbr': 'MEM',
        'city': 'Memphis',
        'code': 'grizzlies',
        'conference': 'Western',
        'displayAbbr': 'MEM',
        'displayConference': 'Western',
        'division': 'Southwest',
        'id': '1610612763',
        'name': 'Grizzlies',
        'color': '182A48',
        'colors': ['182A48', '4C78AD', 'FEA927', 'AAC8E5']
    }, 'MIA': {
        'abbr': 'MIA',
        'city': 'Miami',
        'code': 'heat',
        'conference': 'Eastern',
        'displayAbbr': 'MIA',
        'displayConference': 'Eastern',
        'division': 'Southeast',
        'id': '1610612748',
        'name': 'Heat',
        'color': '98002E',
        'colors': ['98002E', 'F88D1D', '000000']
    }, 'MIL': {
        'abbr': 'MIL',
        'city': 'Milwaukee',
        'code': 'bucks',
        'conference': 'Eastern',
        'displayAbbr': 'MIL',
        'displayConference': 'Eastern',
        'division': 'Central',
        'id': '1610612749',
        'name': 'Bucks',
        'color': 'C41230',
        'colors': ['C41230', '003815', 'BAC4CA']
    }, 'MIN': {
        'abbr': 'MIN',
        'city': 'Minnesota',
        'code': 'timberwolves',
        'conference': 'Western',
        'displayAbbr': 'MIN',
        'displayConference': 'Western',
        'division': 'Northwest',
        'id': '1610612750',
        'name': 'Timberwolves',
        'color': '#003F70',
        'colors': ['003F70', '006F42', 'BAC4CA', 'FFE211', 'DE2032', '000000']
    }, 'NOP': {
        'abbr': 'NOP',
        'city': 'New Orleans',
        'code': 'pelicans',
        'conference': 'Western',
        'displayAbbr': 'NOP',
        'displayConference': 'Western',
        'division': 'Southwest',
        'id': '1610612740',
        'name': 'Pelicans',
        'color': '#002B5C',
        'colors': ['002B5C', 'B4975A', 'E13A3E']
    }, 'NYK': {
        'abbr': 'NYK',
        'city': 'New York',
        'code': 'knicks',
        'conference': 'Eastern',
        'displayAbbr': 'NYK',
        'displayConference': 'Eastern',
        'division': 'Atlantic',
        'id': '1610612752',
        'name': 'Knicks',
        'color': 'F3571F',
        'colors': ['F3571F', '0067B2', 'BAC4CA']
    }, 'OKC': {
        'abbr': 'OKC',
        'city': 'Oklahoma City',
        'code': 'thunder',
        'conference': 'Western',
        'displayAbbr': 'OKC',
        'displayConference': 'Western',
        'division': 'Northwest',
        'id': '1610612760',
        'name': 'Thunder',
        'color': 'FDBB30',
        'colors': ['FDBB30', 'F05133', '007DC3', '002D62']
    }, 'ORL': {
        'abbr': 'ORL',
        'city': 'Orlando',
        'code': 'magic',
        'conference': 'Eastern',
        'displayAbbr': 'ORL',
        'displayConference': 'Eastern',
        'division': 'Southeast',
        'id': '1610612753',
        'name': 'Magic',
        'color': '006BB7',
        'colors': ['006BB7', 'BAC4CA', '000000']
    }, 'PHI': {
        'abbr': 'PHI',
        'city': 'Philadelphia',
        'code': 'sixers',
        'conference': 'Eastern',
        'displayAbbr': 'PHI',
        'displayConference': 'Eastern',
        'division': 'Atlantic',
        'id': '1610612755',
        'name': 'Sixers',
        'color': 'EC003D',
        'colors': ['EC003D', '00559A', 'BAC4CA']
    }, 'PHX': {
        'abbr': 'PHX',
        'city': 'Phoenix',
        'code': 'suns',
        'conference': 'Western',
        'displayAbbr': 'PHX',
        'displayConference': 'Western',
        'division': 'Pacific',
        'id': '1610612756',
        'name': 'Suns',
        'color': 'E45F1F',
        'colors': ['E45F1F', 'F89F1B', 'BAC4CA', '000000']
    }, 'POR': {
        'abbr': 'POR',
        'city': 'Portland',
        'code': 'blazers',
        'conference': 'Western',
        'displayAbbr': 'POR',
        'displayConference': 'Western',
        'division': 'Northwest',
        'id': '1610612757',
        'name': 'Trail Blazers',
        'color': 'DE2032',
        'colors': ['DE2032', 'BAC4CA', '000000']
    }, 'SAC': {
        'abbr': 'SAC',
        'city': 'Sacramento',
        'code': 'kings',
        'conference': 'Western',
        'displayAbbr': 'SAC',
        'displayConference': 'Western',
        'division': 'Pacific',
        'id': '1610612758',
        'name': 'Kings',
        'color': '542E91',
        'colors': ['542E91', 'BAC4CA', '000000']
    }, 'SAS': {
        'abbr': 'SAS',
        'city': 'San Antonio',
        'code': 'spurs',
        'conference': 'Western',
        'displayAbbr': 'SAS',
        'displayConference': 'Western',
        'division': 'Southwest',
        'id': '1610612759',
        'name': 'Spurs',
        'color': '#BA24CA',
        'colors': ['BA24CA', '000000']
    }, 'TOR': {
        'abbr': 'TOR',
        'city': 'Toronto',
        'code': 'raptors',
        'conference': 'Eastern',
        'displayAbbr': 'TOR',
        'displayConference': 'Eastern',
        'division': 'Atlantic',
        'id': '1610612761',
        'name': 'Raptors',
        'color': 'C60033',
        'colors': ['C60033', 'BAC4CA']
    }, 'UTA': {
        'abbr': 'UTA',
        'city': 'Utah',
        'code': 'jazz',
        'conference': 'Western',
        'displayAbbr': 'UTA',
        'displayConference': 'Western',
        'division': 'Northwest',
        'id': '1610612762',
        'name': 'Jazz',
        'color': '#002A5C',
        'colors': ['002A5C', '004812', 'FCB034', 'BACA4CA']
    }, 'WAS': {
        'abbr': 'WAS',
        'city': 'Washington',
        'code': 'wizards',
        'conference': 'Eastern',
        'displayAbbr': 'WAS',
        'displayConference': 'Eastern',
        'division': 'Southeast',
        'id': '1610612764',
        'name': 'Wizards',
        'color': '002A5B',
        'colors': ['002A5B', 'E21836', 'BAC4CA']
    }
}


class _DefaultN:
    Default = 'N'


class _DefaultBlank:
    Default = ''


class _DefaultZero:
    Default = '0'


class League:
    NBA = '00'
    Default = NBA


class PerMode:
    Totals = 'Totals'
    PerGame = 'PerGame'
    MinutesPer = 'MinutesPer'
    Per48 = 'Per48'
    Per40 = 'Per40'
    Per36 = 'Per36'
    PerMinute = 'PerMinute'
    PerPossession = 'PerPossession'
    PerPlay = 'PerPlay'
    Per100Possessions = 'Per100Possessions'
    Per100Plays = 'Per100Plays'
    Default = PerGame


class SeasonType:
    Regular = 'Regular Season'
    Playoffs = 'Playoffs'
    Default = Regular


class MeasureType:
    Base = 'Base'
    Advanced = 'Advanced'
    Misc = 'Misc'
    FourFactors = 'Four Factors'
    Scoring = 'Scoring'
    Opponent = 'Opponent'
    Usage = 'Usage'
    Default = Base


class GroupQuantity:
    Default = 5


class Outcome(_DefaultBlank):
    Win = 'W'
    Loss = 'L'


class Location(_DefaultBlank):
    Home = 'Home'
    Away = 'Away'


class SeasonSegment(_DefaultBlank):
    EntireSeason = ''
    PreAllStar = 'Pre All-Star'
    PostAllStar = 'Post All-Star'


class DateFrom(_DefaultBlank):
    pass


class DateTo(_DefaultBlank):
    pass


class VsConference(_DefaultBlank):
    All = ''
    East = 'East'
    West = 'West'


class VsDivision(_DefaultBlank):
    All = ''
    Atlantic = 'Atlantic'
    Central = 'Central'
    Northwest = 'Northwest'
    Pacific = 'Pacific'
    Southeast = 'Southeast'
    Southwest = 'Southwest'


class GameSegment(_DefaultBlank):
    EntireGame = ''
    FirstHalf = 'First Half'
    SecondHalf = 'Second Half'
    Overtime = 'Overtime'


class ShotClockRange(_DefaultBlank):
    AllRanges = ''
    # I honestly don't know anytime the shot clock would be off
    ShotClockOff = 'ShotClock Off'

    def get(self, n):
        if n > 24 or n < 0:
            return ''
        elif 22 <= n <= 24:
            return '24-22'
        elif 18 <= n < 22:
            return '22-18 Very Early'
        elif 15 <= n < 18:
            return '18-15 Early'
        elif 7 <= n < 15:
            return '15-7 Average'
        elif 4 <= n < 7:
            return '7-4 Late'
        elif 0 <= n < 4:
            return '4-0 Very Late'


class PlusMinus(_DefaultN):
    pass


class PaceAdjust(_DefaultN):
    pass


class Rank(_DefaultN):
    pass


class OpponentTeamID(_DefaultZero):
    pass


class Period(_DefaultZero):
    AllQuarters = '0'
    FirstQuarter = '1'
    SecondQuarter = '2'
    ThirdQuarter = '3'
    FourthQuarter = '4'

    def Overtime(self, n):
        return str(4 + n)


class LastNGames(_DefaultZero):
    pass


class PlayoffRound(_DefaultZero):
    All = '0'
    QuarterFinals = '1'
    SemiFinals = '2'
    ConferenceFinals = '3'
    Finals = '4'


class Month(_DefaultZero):
    All = '0'
    October = '1'
    Novemeber = '2'
    December = '3'
    January = '4'
    February = '5'
    March = '6'
    April = '7'
    May = '8'
    June = '9'
    July = '10'
    August = '11'
    September = '12'


class RangeType(_DefaultZero):
    pass


class StartRange(_DefaultZero):
    pass


class EndRange(_DefaultZero):
    pass


class StartPeriod(Period):
    pass


class EndPeriod(Period):
    pass


class StatCategory:
    PTS = 'PTS'
    FGM = 'FGM'
    FGA = 'FGA'
    FG_PCT = 'FG%'
    FG3M = '3PM'
    FG3A = '3PA'
    FG3_PCT = '3P%'
    FTM = 'FTM'
    FTA = 'FTA'
    FT_PCT = 'FT%'
    OREB = 'OREB'
    DREB = 'DREB'
    REB = 'REB'
    AST = 'AST'
    STL = 'STL'
    BLK = 'BLK'
    TOV = 'TOV'
    EFF = 'EFF'
    AST_TOV = 'AST/TO'
    STL_TOV = 'STL/TOV'
    PF = 'PF'
    Default = PTS


class Scope:
    AllPlayers = 'S'
    Rookies = 'Rookies'
    Default = AllPlayers


class PlayerScope:
    # ugh this is so similar to Scope, why does it have its own
    AllPlayers = 'All Players'
    Rookies = 'Rookie'
    Default = AllPlayers


class PlayerOrTeam:
    Player = 'Player'
    Team = 'Team'
    Default = Player


class GameScope:
    Season = 'Season'
    Last10 = 'Last 10'
    Yesterday = 'Yesterday'
    Finals = 'Finals'
    Default = Season


class Game_Scope(_DefaultBlank):
    Last10 = 'Last 10'
    Yesterday = 'Yesterday'


class Player_or_Team:
    Player = 'P'
    Team = 'T'
    Default = Player


class Conference(VsConference):
    pass


class Division(VsDivision):
    pass


class TeamID(_DefaultZero):
    pass


class PlayerExperience(_DefaultBlank):
    Rookie = 'Rookie'
    Sophomore = 'Sophomore'
    Veteran = 'Veteran'


class PlayerPosition(_DefaultBlank):
    Forward = 'F'
    Center = 'C'
    Guard = 'G'


class StarterBench(_DefaultBlank):
    Starters = 'Starters'
    Bench = 'Bench'


class DraftYear(_DefaultBlank):
    pass


class DraftPick(_DefaultBlank):
    FirstRound = '1st+Round'
    SecondRound = '2nd+Round'
    FirstPick = '1st+Pick'
    Lottery = 'Lottery+Pick'
    Top5 = 'Top+5+Pick'
    Top10 = 'Top+10+Pick'
    Top15 = 'Top+15+Pick'
    Top20 = 'Top+20+Pick'
    Top25 = 'Top+25+Pick'
    Picks11Thru20 = 'Picks+11+Thru+20'
    Picks21Thru30 = 'Picks+21+Thru+30'
    Undrafted = 'Undrafted'


class College(_DefaultBlank):
    pass


class Country(_DefaultBlank):
    pass


class Height(_DefaultBlank):
    '''
    Example:
    for greater than 6ft8 api call should be GT+6-8
    for lower than 7ft3 api call should be LT+7-3
    '''


class Weight(_DefaultBlank):
    '''
    Example:
    for greater than 225lbs api call should be GT+225lbs
    '''


class Counter:
    Default = '1000'


class Sorter:
    PTS = 'PTS'
    FGM = 'FGM'
    FGA = 'FGA'
    FG_PCT = 'FG_PCT'
    FG3M = 'FG3M'
    FG3A = 'FG3A'
    FG3_PCT = 'FG3_PCT'
    FTM = 'FTM'
    FTA = 'FTA'
    FT_PCT = 'FT_PCT'
    OREB = 'OREB'
    DREB = 'DREB'
    AST = 'AST'
    STL = 'STL'
    BLK = 'BLK'
    TOV = 'TOV'
    REB = 'REB'
    Default = PTS


class Direction:
    DESC = 'DESC'
    ASC = 'ASC'
    Default = DESC

In [28]:
from nba_py import _api_scrape, _get_json
from nba_py.constants import *


class Boxscore:
    _endpoint = 'boxscore'

    def __init__(self,
                 game_id,
                 range_type=RangeType.Default,
                 start_period=StartPeriod.Default,
                 end_period=EndPeriod.Default,
                 start_range=StartRange.Default,
                 end_range=EndRange.Default):
        self.json = _get_json(endpoint=self._endpoint,
                              params={'GameID': game_id,
                                      'RangeType': range_type,
                                      'StartPeriod': start_period,
                                      'EndPeriod': end_period,
                                      'StartRange': start_range,
                                      'EndRange': end_range})

    def game_summary(self):
        return _api_scrape(self.json, 0)

    def line_score(self):
        return _api_scrape(self.json, 1)

    def season_series(self):
        return _api_scrape(self.json, 2)

    def last_meeting(self):
        return _api_scrape(self.json, 3)

    def player_stats(self):
        return _api_scrape(self.json, 4)

    def team_stats(self):
        return _api_scrape(self.json, 5)

    def other_stats(self):
        return _api_scrape(self.json, 6)

    def officials(self):
        return _api_scrape(self.json, 7)

    def game_info(self):
        return _api_scrape(self.json, 8)

    def inactive_players(self):
        return _api_scrape(self.json, 9)

    def available_video(self):
        return _api_scrape(self.json, 10)

    def player_track(self):
        return _api_scrape(self.json, 11)

    def player_track_team(self):
        return _api_scrape(self.json, 12)


class BoxscoreScoring(Boxscore):
    _endpoint = 'boxscorescoring'

    def sql_players_scoring(self):
        return _api_scrape(self.json, 13)

    def sql_team_scoring(self):
        return _api_scrape(self.json, 14)


class BoxscoreUsage(Boxscore):
    _endpoint = 'boxscoreusage'

    def sql_players_usage(self):
        return _api_scrape(self.json, 13)

    def sql_team_usage(self):
        return _api_scrape(self.json, 14)


class BoxscoreMisc(Boxscore):
    _endpoint = 'boxscoremisc'

    def sql_players_misc(self):
        return _api_scrape(self.json, 13)

    def sql_team_misc(self):
        return _api_scrape(self.json, 14)


class BoxscoreAdvanced(Boxscore):
    _endpoint = 'boxscoreadvanced'

    def sql_players_advanced(self):
        return _api_scrape(self.json, 13)

    def sql_team_advanced(self):
        return _api_scrape(self.json, 14)


class BoxscoreFourFactors(Boxscore):
    _endpoint = 'boxscorefourfactors'

    def sql_players_four_factors(self):
        return _api_scrape(self.json, 13)

    def sql_team_four_factors(self):
        return _api_scrape(self.json, 14)


class PlayByPlay:
    _endpoint = 'playbyplay'

    def __init__(self,
                 game_id,
                 start_period=StartPeriod.Default,
                 end_period=EndPeriod.Default):
        self.json = _get_json(endpoint=self._endpoint,
                              params={'GameID': game_id,
                                      'StartPeriod': start_period,
                                      'EndPeriod': end_period})

    def info(self):
        return _api_scrape(self.json, 0)

    def available_video(self):
        return _api_scrape(self.json, 1)

In [29]:
from nba_py import _api_scrape, _get_json
from nba_py.constants import *


class Leaders:
    _endpoint = 'leagueleaders'

    def __init__(self,
                 league_id=League.Default,
                 per_mode=PerMode.Default,
                 stat_category=StatCategory.Default,
                 season=CURRENT_SEASON,
                 season_type=SeasonType.Default,
                 scope=Scope.Default,):
        self.json = _get_json(endpoint=self._endpoint,
                              params={'LeagueID': league_id,
                                      'PerMode': per_mode,
                                      'StatCategory': stat_category,
                                      'Season': season,
                                      'SeasonType': season_type,
                                      'Scope': scope})

    def results(self):
        return _api_scrape(self.json, 0)


class LeadersTiles:
    _endpoint = 'leaderstiles'

    def __init__(self,
                 league_id=League.Default,
                 season=CURRENT_SEASON,
                 season_type=SeasonType.Default,
                 game_scope=GameScope.Default,
                 player_scope=PlayerScope.Default,
                 player_or_team=PlayerOrTeam.Default,
                 stat_category=StatCategory.Default):
        self.json = _get_json(endpoint=self._endpoint,
                              params={'LeagueID': league_id,
                                      'Stat': stat_category,
                                      'Season': season,
                                      'SeasonType': season_type,
                                      'GameScope': game_scope,
                                      'PlayerScope': player_scope,
                                      'PlayerOrTeam': player_or_team})

    def current_season_high(self):
        return _api_scrape(self.json, 0)

    def alltime_season_high(self):
        return _api_scrape(self.json, 1)

    def last_season_high(self):
        return _api_scrape(self.json, 2)

    def low_season_high(self):
        return _api_scrape(self.json, 3)


class Lineups:
    _endpoint = 'leaguedashlineups'

    def __init__(self,
                 group_quantity=GroupQuantity.Default,
                 season_type=SeasonType.Default,
                 measure_type=MeasureType.Default,
                 per_mode=PerMode.Default,
                 plus_minus=PlusMinus.Default,
                 pace_adjust=PaceAdjust.Default,
                 rank=Rank.Default,
                 season=CURRENT_SEASON,
                 outcome=Outcome.Default,
                 location=Location.Default,
                 month=Month.Default,
                 season_segment=SeasonSegment.Default,
                 date_from=DateFrom.Default,
                 date_to=DateTo.Default,
                 opponent_team_id=OpponentTeamID.Default,
                 vs_conference=VsConference.Default,
                 vs_division=VsDivision.Default,
                 game_segment=GameSegment.Default,
                 period=Period.Default,
                 last_n_games=LastNGames.Default):
        self.json = _get_json(endpoint=self._endpoint,
                              params={'GroupQuantity': group_quantity,
                                      'SeasonType': season_type,
                                      'MeasureType': measure_type,
                                      'PerMode': per_mode,
                                      'PlusMinus': plus_minus,
                                      'PaceAdjust': pace_adjust,
                                      'Rank': rank,
                                      'Season': season,
                                      'Outcome': outcome,
                                      'Location': location,
                                      'Month': month,
                                      'SeasonSegment': season_segment,
                                      'DateFrom': date_from,
                                      'DateTo': date_to,
                                      'OpponentTeamID': opponent_team_id,
                                      'VsConference': vs_conference,
                                      'VsDivision': vs_division,
                                      'GameSegment': game_segment,
                                      'Period': period,
                                      'LastNGames': last_n_games})

    def overall(self):
        return _api_scrape(self.json, 0)

class PlayerStats:
    _endpoint = 'leaguedashplayerstats'

    def __init__(self,
                 season_type=SeasonType.Default,
                 measure_type=MeasureType.Default,
                 per_mode=PerMode.Default,
                 plus_minus=PlusMinus.Default,
                 pace_adjust=PaceAdjust.Default,
                 rank=Rank.Default,
                 season=CURRENT_SEASON,
                 playoff_round=PlayoffRound.Default,
                 outcome=Outcome.Default,
                 location=Location.Default,
                 month=Month.Default,
                 season_segment=SeasonSegment.Default,
                 date_from=DateFrom.Default,
                 date_to=DateTo.Default,
                 opponent_team_id=OpponentTeamID.Default,
                 vs_conference=VsConference.Default,
                 vs_division=VsDivision.Default,
                 team_id=TeamID.Default,
                 conference=Conference.Default,
                 division=Division.Default,
                 game_segment=GameSegment.Default,
                 period=Period.Default,
                 shot_clock_range=ShotClockRange.Default,
                 last_n_games=LastNGames.Default,
                 game_scope=Game_Scope.Default,
                 player_experience=PlayerExperience.Default,
                 player_position=PlayerPosition.Default,
                 starter_bench=StarterBench.Default,
                 draft_year=DraftYear.Default,
                 draft_pick=DraftPick.Default,
                 college=College.Default,
                 country=Country.Default,
                 height=Height.Default,
                 weight=Weight.Default
                 ):
        self.json = _get_json(endpoint=self._endpoint,
                              params={'SeasonType': season_type,
                                      'MeasureType': measure_type,
                                      'PerMode': per_mode,
                                      'PlusMinus': plus_minus,
                                      'PaceAdjust': pace_adjust,
                                      'Rank': rank,
                                      'Season': season,
                                      'PORound': playoff_round,
                                      'Outcome': outcome,
                                      'Location': location,
                                      'Month': month,
                                      'SeasonSegment': season_segment,
                                      'DateFrom': date_from,
                                      'DateTo': date_to,
                                      'OpponentTeamID': opponent_team_id,
                                      'VsConference': vs_conference,
                                      'VsDivision': vs_division,
                                      'TeamID': team_id,
                                      'Conference': conference,
                                      'Division': division,
                                      'GameSegment': game_segment,
                                      'Period': period,
                                      'ShotClockRange': shot_clock_range,
                                      'LastNGames': last_n_games,
                                      'GameScope': game_scope,
                                      'PlayerExperience': player_experience,
                                      'PlayerPosition': player_position,
                                      'StarterBench': starter_bench,
                                      'DraftYear': draft_year,
                                      'DraftPick': draft_pick,
                                      'College': college,
                                      'Country': country,
                                      'Height': height,
                                      'Weight': weight
                                      })

    def overall(self):
        return _api_scrape(self.json, 0)

class GameLog:
    _endpoint = 'leaguegamelog'

    def __init__(self,
                 league_id=League.Default,
                 season=CURRENT_SEASON,
                 season_type=SeasonType.Default,
                 player_or_team=Player_or_Team.Default,
                 counter=Counter.Default,
                 sorter=Sorter.Default,
                 direction=Direction.Default,
                 ):

        self.json = _get_json(endpoint=self._endpoint,
                              params={'LeagueID': league_id,
                                      'Season': season,
                                      'SeasonType': season_type,
                                      'PlayerOrTeam': player_or_team,
                                      'Counter': counter,
                                      'Sorter': sorter,
                                      'Direction': direction
                                      })

    def overall(self):
        return _api_scrape(self.json, 0)

In [30]:
from nba_py import _api_scrape, _get_json, HAS_PANDAS
from nba_py.constants import *


class PlayerNotFoundException(Exception):
    pass


def get_player(first_name,
               last_name,
               season=CURRENT_SEASON,
               only_current=0,
               just_id=True):
    """
    Calls our PlayerList class to get a full list of players and then returns
    just an id if specified or the full row of player information

    Args:
        :first_name: First name of the player
        :last_name: Last name of the player
        :only_current: Only wants the current list of players
        :just_id: Only wants the id of the player

    Returns:
        Either the ID or full row of information of the player inputted

    Raises:
        :PlayerNotFoundException::
    """
    name = '{}, {}'.format(last_name, first_name).lower()
    pl = PlayerList(season=season, only_current=only_current).info()
    hdr = 'DISPLAY_LAST_COMMA_FIRST'
    if HAS_PANDAS:
        item = pl[pl.DISPLAY_LAST_COMMA_FIRST.str.lower() == name]
    else:
        item = next(plyr for plyr in pl if str(plyr[hdr]).lower() == name)
    if len(item) == 0:
        raise PlayerNotFoundException
    elif just_id:
        return item['PERSON_ID']
    else:
        return item


class PlayerList:
    """
    Contains a list of all players for a season, if specified, and will only
    contain current players if specified as well

    Args:
        :league_id: ID for the league to look in (Default is 00)
        :season: Season given to look up
        :only_current: Restrict lookup to only current players

    Attributes:
        :json: Contains the full json dump to play around with
    """
    _endpoint = 'commonallplayers'

    def __init__(self,
                 league_id=League.NBA,
                 season=CURRENT_SEASON,
                 only_current=1):
        self.json = _get_json(endpoint=self._endpoint,
                              params={'LeagueID': league_id,
                                      'Season': season,
                                      'IsOnlyCurrentSeason': only_current})

    def info(self):
        return _api_scrape(self.json, 0)


class PlayerSummary:
    """
    Contains common player information like headline stats, weight, etc.

    Args:
        :player_id: ID of the player to look up

    Attributes:
        :json: Contains the full json dump to play around with
    """
    _endpoint = 'commonplayerinfo'

    def __init__(self,
                 player_id):
        self.json = _get_json(endpoint=self._endpoint,
                              params={'PlayerID': player_id})

    def info(self):
        return _api_scrape(self.json, 0)

    def headline_stats(self):
        return _api_scrape(self.json, 1)


class _PlayerDashboard:
    """
    Has all the basic arguments for all of the Player Dashboard type objects

    Args:
        :player_id: ID of the player to look up
        :team_id: ID of the team to look up
        :measure_type: Specifies type of measure to use (Base, Advanced, etc.)
        :per_mode: Mode to measure statistics (Totals, PerGame, Per36, etc.)
        :plus_minus: Whether or not to consider plus minus (Y or N)
        :pace_adjust: Whether or not to pace adjust stats (Y or N)
        :rank: Whether or not to consider rank (Y or N)
        :league_id: ID for the league to look in (Default is 00)
        :season: Season given to look up
        :season_type: Season type to consider (Regular / Playoffs)
        :po_round: Playoff round
        :outcome: Filter out by wins or losses
        :location: Filter out by home or away
        :month: Specify month to filter by
        :season_segment: Filter by pre/post all star break
        :date_from: Filter out games before a specific date
        :date_to: Filter out games after a specific date
        :opponent_team_id: Opponent team ID to look up
        :vs_conference: Filter by conference
        :vs_division: Filter by division
        :game_segment: Filter by half / overtime
        :period: Filter by quarter / specific overtime
        :shot_clock_range: Filter statistics by range in shot clock
        :last_n_games: Filter by number of games specified in N

    Attributes:
        :json: Contains the full json dump to play around with
    """
    _endpoint = ''

    def __init__(self,
                 player_id,
                 team_id=0,
                 measure_type=MeasureType.Default,
                 per_mode=PerMode.Default,
                 plus_minus=PlusMinus.Default,
                 pace_adjust=PaceAdjust.Default,
                 rank=PaceAdjust.Default,
                 league_id=League.Default,
                 season=CURRENT_SEASON,
                 season_type=SeasonType.Default,
                 po_round=PlayoffRound.Default,
                 outcome=Outcome.Default,
                 location=Location.Default,
                 month=Month.Default,
                 season_segment=SeasonSegment.Default,
                 date_from=DateFrom.Default,
                 date_to=DateTo.Default,
                 opponent_team_id=OpponentTeamID.Default,
                 vs_conference=VsConference.Default,
                 vs_division=VsDivision.Default,
                 game_segment=GameSegment.Default,
                 period=Period.Default,
                 shot_clock_range=ShotClockRange.Default,
                 last_n_games=LastNGames.Default):
        self.json = _get_json(endpoint=self._endpoint,
                              params={'PlayerID': player_id,
                                      'TeamID': team_id,
                                      'MeasureType': measure_type,
                                      'PerMode': per_mode,
                                      'PlusMinus': plus_minus,
                                      'PaceAdjust': pace_adjust,
                                      'Rank': rank,
                                      'LeagueID': league_id,
                                      'Season': season,
                                      'SeasonType': season_type,
                                      'PORound': po_round,
                                      'Outcome': outcome,
                                      'Location': location,
                                      'Month': month,
                                      'SeasonSegment': season_segment,
                                      'DateFrom': date_from,
                                      'DateTo': date_to,
                                      'OpponentTeamID': opponent_team_id,
                                      'VsConference': vs_conference,
                                      'VsDivision': vs_division,
                                      'GameSegment': game_segment,
                                      'Period': period,
                                      'ShotClockRange': shot_clock_range,
                                      'LastNGames': last_n_games})

    def overall(self):
        return _api_scrape(self.json, 0)


class PlayerGeneralSplits(_PlayerDashboard):
    """

    UNFORTUNATELY MARKED AS FORBIDDEN BY THE NBA

    Contains stats pertaining to location, wins and losses, pre/post all star
    break, starting position, and numbers of days rest

    Args:
        :player_id: ID of the player to look up
        :team_id: ID of the team to look up
        :measure_type: Specifies type of measure to use (Base, Advanced, etc.)
        :per_mode: Mode to measure statistics (Totals, PerGame, Per36, etc.)
        :plus_minus: Whether or not to consider plus minus (Y or N)
        :pace_adjust: Whether or not to pace adjust stats (Y or N)
        :rank: Whether or not to consider rank (Y or N)
        :league_id: ID for the league to look in (Default is 00)
        :season: Season given to look up
        :season_type: Season type to consider (Regular / Playoffs)
        :po_round: Playoff round
        :outcome: Filter out by wins or losses
        :location: Filter out by home or away
        :month: Specify month to filter by
        :season_segment: Filter by pre/post all star break
        :date_from: Filter out games before a specific date
        :date_to: Filter out games after a specific date
        :opponent_team_id: Opponent team ID to look up
        :vs_conference: Filter by conference
        :vs_division: Filter by division
        :game_segment: Filter by half / overtime
        :period: Filter by quarter / specific overtime
        :shot_clock_range: Filter statistics by range in shot clock
        :last_n_games: Filter by number of games specified in N

    Attributes:
        :json: Contains the full json dump to play around with
    """
    _endpoint = 'playerdashboardbygeneralsplits'

    def location(self):
        return _api_scrape(self.json, 1)

    def win_losses(self):
        return _api_scrape(self.json, 2)

    def pre_post_all_star(self):
        return _api_scrape(self.json, 3)

    def starting_position(self):
        return _api_scrape(self.json, 4)

    def days_rest(self):
        return _api_scrape(self.json, 5)


class PlayerOpponentSplits(_PlayerDashboard):
    """

    UNFORTUNATELY MARKED AS FORBIDDEN BY THE NBA

    Contains stats pertaining to player stats vs certain opponents by division,
    conference, and by specific team opponent

    Args:
        :player_id: ID of the player to look up
        :team_id: ID of the team to look up
        :measure_type: Specifies type of measure to use (Base, Advanced, etc.)
        :per_mode: Mode to measure statistics (Totals, PerGame, Per36, etc.)
        :plus_minus: Whether or not to consider plus minus (Y or N)
        :pace_adjust: Whether or not to pace adjust stats (Y or N)
        :rank: Whether or not to consider rank (Y or N)
        :league_id: ID for the league to look in (Default is 00)
        :season: Season given to look up
        :season_type: Season type to consider (Regular / Playoffs)
        :po_round: Playoff round
        :outcome: Filter out by wins or losses
        :location: Filter out by home or away
        :month: Specify month to filter by
        :season_segment: Filter by pre/post all star break
        :date_from: Filter out games before a specific date
        :date_to: Filter out games after a specific date
        :opponent_team_id: Opponent team ID to look up
        :vs_conference: Filter by conference
        :vs_division: Filter by division
        :game_segment: Filter by half / overtime
        :period: Filter by quarter / specific overtime
        :shot_clock_range: Filter statistics by range in shot clock
        :last_n_games: Filter by number of games specified in N

    Attributes:
        :json: Contains the full json dump to play around with
    """
    _endpoint = 'playerdashboardbyopponent'

    def by_conference(self):
        return _api_scrape(self.json, 1)

    def by_division(self):
        return _api_scrape(self.json, 2)

    def by_opponent(self):
        return _api_scrape(self.json, 3)


class PlayerLastNGamesSplits(_PlayerDashboard):
    """
    Contains players stats per last 5, 10, 15, and 20 games, or specified
    number of games.

    Args:
        :player_id: ID of the player to look up
        :team_id: ID of the team to look up
        :measure_type: Specifies type of measure to use (Base, Advanced, etc.)
        :per_mode: Mode to measure statistics (Totals, PerGame, Per36, etc.)
        :plus_minus: Whether or not to consider plus minus (Y or N)
        :pace_adjust: Whether or not to pace adjust stats (Y or N)
        :rank: Whether or not to consider rank (Y or N)
        :league_id: ID for the league to look in (Default is 00)
        :season: Season given to look up
        :season_type: Season type to consider (Regular / Playoffs)
        :po_round: Playoff round
        :outcome: Filter out by wins or losses
        :location: Filter out by home or away
        :month: Specify month to filter by
        :season_segment: Filter by pre/post all star break
        :date_from: Filter out games before a specific date
        :date_to: Filter out games after a specific date
        :opponent_team_id: Opponent team ID to look up
        :vs_conference: Filter by conference
        :vs_division: Filter by division
        :game_segment: Filter by half / overtime
        :period: Filter by quarter / specific overtime
        :shot_clock_range: Filter statistics by range in shot clock
        :last_n_games: Filter by number of games specified in N

    Attributes:
        :json: Contains the full json dump to play around with
    """
    _endpoint = 'playerdashboardbylastngames'

    def last5(self):
        return _api_scrape(self.json, 1)

    def last10(self):
        return _api_scrape(self.json, 2)

    def last15(self):
        return _api_scrape(self.json, 3)

    def last20(self):
        return _api_scrape(self.json, 4)

    def gamenumber(self):
        return _api_scrape(self.json, 5)


class PlayerInGameSplits(_PlayerDashboard):
    """
    Contains player stats by half, by quarter, by score margin, and by actual
    margins.

    Args:
        :player_id: ID of the player to look up
        :team_id: ID of the team to look up
        :measure_type: Specifies type of measure to use (Base, Advanced, etc.)
        :per_mode: Mode to measure statistics (Totals, PerGame, Per36, etc.)
        :plus_minus: Whether or not to consider plus minus (Y or N)
        :pace_adjust: Whether or not to pace adjust stats (Y or N)
        :rank: Whether or not to consider rank (Y or N)
        :league_id: ID for the league to look in (Default is 00)
        :season: Season given to look up
        :season_type: Season type to consider (Regular / Playoffs)
        :po_round: Playoff round
        :outcome: Filter out by wins or losses
        :location: Filter out by home or away
        :month: Specify month to filter by
        :season_segment: Filter by pre/post all star break
        :date_from: Filter out games before a specific date
        :date_to: Filter out games after a specific date
        :opponent_team_id: Opponent team ID to look up
        :vs_conference: Filter by conference
        :vs_division: Filter by division
        :game_segment: Filter by half / overtime
        :period: Filter by quarter / specific overtime
        :shot_clock_range: Filter statistics by range in shot clock
        :last_n_games: Filter by number of games specified in N

    Attributes:
        :json: Contains the full json dump to play around with
    """
    _endpoint = 'playerdashboardbygamesplits'

    def by_half(self):
        return _api_scrape(self.json, 1)

    def by_period(self):
        return _api_scrape(self.json, 2)

    def by_score_margin(self):
        return _api_scrape(self.json, 3)

    def by_actual_margin(self):
        return _api_scrape(self.json, 4)


class PlayerClutchSplits(_PlayerDashboard):
    """
    Contains a lot of methods for last n minutes with a deficit of x points

    Args:
        :player_id: ID of the player to look up
        :team_id: ID of the team to look up
        :measure_type: Specifies type of measure to use (Base, Advanced, etc.)
        :per_mode: Mode to measure statistics (Totals, PerGame, Per36, etc.)
        :plus_minus: Whether or not to consider plus minus (Y or N)
        :pace_adjust: Whether or not to pace adjust stats (Y or N)
        :rank: Whether or not to consider rank (Y or N)
        :league_id: ID for the league to look in (Default is 00)
        :season: Season given to look up
        :season_type: Season type to consider (Regular / Playoffs)
        :po_round: Playoff round
        :outcome: Filter out by wins or losses
        :location: Filter out by home or away
        :month: Specify month to filter by
        :season_segment: Filter by pre/post all star break
        :date_from: Filter out games before a specific date
        :date_to: Filter out games after a specific date
        :opponent_team_id: Opponent team ID to look up
        :vs_conference: Filter by conference
        :vs_division: Filter by division
        :game_segment: Filter by half / overtime
        :period: Filter by quarter / specific overtime
        :shot_clock_range: Filter statistics by range in shot clock
        :last_n_games: Filter by number of games specified in N

    Attributes:
        :json: Contains the full json dump to play around with
    """
    _endpoint = 'playerdashboardbyclutch'

    def last5min_deficit_5point(self):
        """
        Results in last 5 minutes <= 5 points
        """
        return _api_scrape(self.json, 1)

    def last3min_deficit_5point(self):
        """
        Results in last 5 minutes <= 5 points
        """
        return _api_scrape(self.json, 2)

    def last1min_deficit_5point(self):
        """
        Results in last 5 minutes <= 5 points
        """
        return _api_scrape(self.json, 3)

    def last30sec_deficit_3point(self):
        """
        Results in last 5 minutes <= 5 points
        """
        return _api_scrape(self.json, 4)

    def last10sec_deficit_3point(self):
        """
        Results in last 5 minutes <= 5 points
        """
        return _api_scrape(self.json, 5)

    def last5min_plusminus_5point(self):
        """
        Last 5 minutes +/= 5 points
        """
        return _api_scrape(self.json, 6)

    def last3min_plusminus_5point(self):
        """
        Last 3 minutes +/= 5 points
        """
        return _api_scrape(self.json, 7)

    def last1min_plusminus_5point(self):
        """
        Last 1 minutes +/= 5 points
        """
        return _api_scrape(self.json, 8)

    def last30sec_plusminus_5point(self):
        """
        Last 30 seconds +/= 3 points
        """
        return _api_scrape(self.json, 9)


class PlayerShootingSplits(_PlayerDashboard):
    """

    UNFORTUNATELY MARKED AS FORBIDDEN BY THE NBA

    Shooting stats based on distance, area, assisted to, shot types, and
    assisted by.

    Args:
        :player_id: ID of the player to look up
        :team_id: ID of the team to look up
        :measure_type: Specifies type of measure to use (Base, Advanced, etc.)
        :per_mode: Mode to measure statistics (Totals, PerGame, Per36, etc.)
        :plus_minus: Whether or not to consider plus minus (Y or N)
        :pace_adjust: Whether or not to pace adjust stats (Y or N)
        :rank: Whether or not to consider rank (Y or N)
        :league_id: ID for the league to look in (Default is 00)
        :season: Season given to look up
        :season_type: Season type to consider (Regular / Playoffs)
        :po_round: Playoff round
        :outcome: Filter out by wins or losses
        :location: Filter out by home or away
        :month: Specify month to filter by
        :season_segment: Filter by pre/post all star break
        :date_from: Filter out games before a specific date
        :date_to: Filter out games after a specific date
        :opponent_team_id: Opponent team ID to look up
        :vs_conference: Filter by conference
        :vs_division: Filter by division
        :game_segment: Filter by half / overtime
        :period: Filter by quarter / specific overtime
        :shot_clock_range: Filter statistics by range in shot clock
        :last_n_games: Filter by number of games specified in N

    Attributes:
        :json: Contains the full json dump to play around with
    """
    _endpoint = 'playerdashboardbyshootingsplits'

    def shot_5ft(self):
        return _api_scrape(self.json, 1)

    def shot_8ft(self):
        return _api_scrape(self.json, 2)

    def shot_areas(self):
        return _api_scrape(self.json, 3)

    def assisted_shots(self):
        return _api_scrape(self.json, 4)

    def shot_types_summary(self):
        return _api_scrape(self.json, 5)

    def shot_types_detail(self):
        return _api_scrape(self.json, 6)

    def assissted_by(self):
        return _api_scrape(self.json, 7)


class PlayerPerformanceSplits(_PlayerDashboard):
    """
    Player stats by different performance metrics such as score differntial,
    points scored, and points scored against

    Args:
        :player_id: ID of the player to look up
        :team_id: ID of the team to look up
        :measure_type: Specifies type of measure to use (Base, Advanced, etc.)
        :per_mode: Mode to measure statistics (Totals, PerGame, Per36, etc.)
        :plus_minus: Whether or not to consider plus minus (Y or N)
        :pace_adjust: Whether or not to pace adjust stats (Y or N)
        :rank: Whether or not to consider rank (Y or N)
        :league_id: ID for the league to look in (Default is 00)
        :season: Season given to look up
        :season_type: Season type to consider (Regular / Playoffs)
        :po_round: Playoff round
        :outcome: Filter out by wins or losses
        :location: Filter out by home or away
        :month: Specify month to filter by
        :season_segment: Filter by pre/post all star break
        :date_from: Filter out games before a specific date
        :date_to: Filter out games after a specific date
        :opponent_team_id: Opponent team ID to look up
        :vs_conference: Filter by conference
        :vs_division: Filter by division
        :game_segment: Filter by half / overtime
        :period: Filter by quarter / specific overtime
        :shot_clock_range: Filter statistics by range in shot clock
        :last_n_games: Filter by number of games specified in N

    Attributes:
        :json: Contains the full json dump to play around with
    """
    _endpoint = 'playerdashboardbyteamperformance'

    def score_differential(self):
        return _api_scrape(self.json, 1)

    def points_scored(self):
        return _api_scrape(self.json, 2)

    def points_against(self):
        return _api_scrape(self.json, 3)


class PlayerYearOverYearSplits(_PlayerDashboard):
    """
    Displays player stats over the given season and over all seasons in the
    given league

    Args:
        :player_id: ID of the player to look up
        :team_id: ID of the team to look up
        :measure_type: Specifies type of measure to use (Base, Advanced, etc.)
        :per_mode: Mode to measure statistics (Totals, PerGame, Per36, etc.)
        :plus_minus: Whether or not to consider plus minus (Y or N)
        :pace_adjust: Whether or not to pace adjust stats (Y or N)
        :rank: Whether or not to consider rank (Y or N)
        :league_id: ID for the league to look in (Default is 00)
        :season: Season given to look up
        :season_type: Season type to consider (Regular / Playoffs)
        :po_round: Playoff round
        :outcome: Filter out by wins or losses
        :location: Filter out by home or away
        :month: Specify month to filter by
        :season_segment: Filter by pre/post all star break
        :date_from: Filter out games before a specific date
        :date_to: Filter out games after a specific date
        :opponent_team_id: Opponent team ID to look up
        :vs_conference: Filter by conference
        :vs_division: Filter by division
        :game_segment: Filter by half / overtime
        :period: Filter by quarter / specific overtime
        :shot_clock_range: Filter statistics by range in shot clock
        :last_n_games: Filter by number of games specified in N

    Attributes:
        :json: Contains the full json dump to play around with
    """
    _endpoint = 'playerdashboardbyyearoveryear'

    def by_year(self):
        return _api_scrape(self.json, 1)


class PlayerCareer:
    """
    Contains stats based on several parameters such as career regular season
    totals, post season career totals, all star season careers totals, college
    season career totals, etc.

    Args:
        :player_id: Player ID to look up
        :per_mode: Mode to measure statistics (Totals, PerGame, Per36, etc.)
        :league_id: ID for the league to look in (Default is 00)

    Attributes:
        :json: Contains the full json dump to play around with
    """
    _endpoint = 'playercareerstats'

    def __init__(self,
                 player_id,
                 per_mode=PerMode.PerGame,
                 league_id=League.NBA):
        self.json = _get_json(endpoint=self._endpoint,
                              params={'PlayerID': player_id,
                                      'LeagueID': league_id,
                                      'PerMode': per_mode})

    def regular_season_totals(self):
        return _api_scrape(self.json, 0)

    def regular_season_career_totals(self):
        return _api_scrape(self.json, 1)

    def post_season_totals(self):
        return _api_scrape(self.json, 2)

    def post_season_career_totals(self):
        return _api_scrape(self.json, 3)

    def all_star_season_totals(self):
        return _api_scrape(self.json, 4)

    def career_all_star_season_totals(self):
        return _api_scrape(self.json, 5)

    def college_season_totals(self):
        return _api_scrape(self.json, 6)

    def college_season_career_totals(self):
        return _api_scrape(self.json, 7)

    def regular_season_rankings(self):
        return _api_scrape(self.json, 8)

    def post_season_rankings(self):
        return _api_scrape(self.json, 9)


class PlayerProfile(PlayerCareer):
    """
    Contains a more in depth version of player career stats with season highs,
    career highs, and when the player's next game is

    Args:
        :player_id: Player ID to look up
        :per_mode: Mode to measure statistics (Totals, PerGame, Per36, etc.)
        :league_id: ID for the league to look in (Default is 00)

    Attributes:
        :json: Contains the full json dump to play around with
    """
    _endpoint = 'playerprofilev2'

    def season_highs(self):
        return _api_scrape(self.json, 10)

    def career_highs(self):
        return _api_scrape(self.json, 11)

    def next_game(self):
        return _api_scrape(self.json, 12)


class PlayerGameLogs:
    """
    Contains a full log of all the games for a player for a given season

    Args:
        :player_id: ID of the player to look up
        :league_id: ID for the league to look in (Default is 00)
        :season: Season given to look up
        :season_type: Season type to consider (Regular / Playoffs)

    Attributes:
        :json: Contains the full json dump to play around with
    """
    _endpoint = 'playergamelog'

    def __init__(self,
                 player_id,
                 league_id=League.NBA,
                 season=CURRENT_SEASON,
                 season_type=SeasonType.Regular):
        self.json = _get_json(endpoint=self._endpoint,
                              params={'PlayerID': player_id,
                                      'LeagueID': league_id,
                                      'Season': season,
                                      'SeasonType': season_type})

    def info(self):
        return _api_scrape(self.json, 0)


class PlayerShotTracking(_PlayerDashboard):
    """

    UNFORTUNATELY MARKED AS FORBIDDEN BY THE NBA

    Tracking data for shooting for a given player

    Args:
        :player_id: ID of the player to look up
        :team_id: ID of the team to look up
        :measure_type: Specifies type of measure to use (Base, Advanced, etc.)
        :per_mode: Mode to measure statistics (Totals, PerGame, Per36, etc.)
        :plus_minus: Whether or not to consider plus minus (Y or N)
        :pace_adjust: Whether or not to pace adjust stats (Y or N)
        :rank: Whether or not to consider rank (Y or N)
        :league_id: ID for the league to look in (Default is 00)
        :season: Season given to look up
        :season_type: Season type to consider (Regular / Playoffs)
        :po_round: Playoff round
        :outcome: Filter out by wins or losses
        :location: Filter out by home or away
        :month: Specify month to filter by
        :season_segment: Filter by pre/post all star break
        :date_from: Filter out games before a specific date
        :date_to: Filter out games after a specific date
        :opponent_team_id: Opponent team ID to look up
        :vs_conference: Filter by conference
        :vs_division: Filter by division
        :game_segment: Filter by half / overtime
        :period: Filter by quarter / specific overtime
        :shot_clock_range: Filter statistics by range in shot clock
        :last_n_games: Filter by number of games specified in N

    Attributes:
        :json: Contains the full json dump to play around with
    """
    _endpoint = 'playerdashptshots'

    def general_shooting(self):
        return _api_scrape(self.json, 1)

    def shot_clock_shooting(self):
        return _api_scrape(self.json, 2)

    def dribble_shooting(self):
        return _api_scrape(self.json, 3)

    def closest_defender_shooting(self):
        return _api_scrape(self.json, 4)

    def closest_defender_shooting_long(self):
        return _api_scrape(self.json, 5)

    def touch_time_shooting(self):
        return _api_scrape(self.json, 6)


class PlayerReboundTracking(_PlayerDashboard):
    """

    UNFORTUNATELY MARKED AS FORBIDDEN BY THE NBA

    Tracking data for rebounding for a given player

    Args:
        :player_id: ID of the player to look up
        :team_id: ID of the team to look up
        :measure_type: Specifies type of measure to use (Base, Advanced, etc.)
        :per_mode: Mode to measure statistics (Totals, PerGame, Per36, etc.)
        :plus_minus: Whether or not to consider plus minus (Y or N)
        :pace_adjust: Whether or not to pace adjust stats (Y or N)
        :rank: Whether or not to consider rank (Y or N)
        :league_id: ID for the league to look in (Default is 00)
        :season: Season given to look up
        :season_type: Season type to consider (Regular / Playoffs)
        :po_round: Playoff round
        :outcome: Filter out by wins or losses
        :location: Filter out by home or away
        :month: Specify month to filter by
        :season_segment: Filter by pre/post all star break
        :date_from: Filter out games before a specific date
        :date_to: Filter out games after a specific date
        :opponent_team_id: Opponent team ID to look up
        :vs_conference: Filter by conference
        :vs_division: Filter by division
        :game_segment: Filter by half / overtime
        :period: Filter by quarter / specific overtime
        :shot_clock_range: Filter statistics by range in shot clock
        :last_n_games: Filter by number of games specified in N

    Attributes:
        :json: Contains the full json dump to play around with
    """
    _endpoint = 'playerdashptreb'

    def shot_type_rebounding(self):
        return _api_scrape(self.json, 1)

    def num_contested_rebounding(self):
        return _api_scrape(self.json, 2)

    def shot_distance_rebounding(self):
        return _api_scrape(self.json, 3)

    def rebound_distance_rebounding(self):
        return _api_scrape(self.json, 4)


class PlayerPassTracking(_PlayerDashboard):
    """
    Tracking data for passing for a given player

    Args:
        :player_id: ID of the player to look up
        :team_id: ID of the team to look up
        :measure_type: Specifies type of measure to use (Base, Advanced, etc.)
        :per_mode: Mode to measure statistics (Totals, PerGame, Per36, etc.)
        :plus_minus: Whether or not to consider plus minus (Y or N)
        :pace_adjust: Whether or not to pace adjust stats (Y or N)
        :rank: Whether or not to consider rank (Y or N)
        :league_id: ID for the league to look in (Default is 00)
        :season: Season given to look up
        :season_type: Season type to consider (Regular / Playoffs)
        :po_round: Playoff round
        :outcome: Filter out by wins or losses
        :location: Filter out by home or away
        :month: Specify month to filter by
        :season_segment: Filter by pre/post all star break
        :date_from: Filter out games before a specific date
        :date_to: Filter out games after a specific date
        :opponent_team_id: Opponent team ID to look up
        :vs_conference: Filter by conference
        :vs_division: Filter by division
        :game_segment: Filter by half / overtime
        :period: Filter by quarter / specific overtime
        :shot_clock_range: Filter statistics by range in shot clock
        :last_n_games: Filter by number of games specified in N

    Attributes:
        :json: Contains the full json dump to play around with
    """
    _endpoint = 'playerdashptpass'

    def passes_made(self):
        return _api_scrape(self.json, 0)

    def passes_recieved(self):
        return _api_scrape(self.json, 1)


class PlayerDefenseTracking(_PlayerDashboard):
    """
    Tracking data for defense for a given player

    Args:
        :player_id: ID of the player to look up
        :team_id: ID of the team to look up
        :measure_type: Specifies type of measure to use (Base, Advanced, etc.)
        :per_mode: Mode to measure statistics (Totals, PerGame, Per36, etc.)
        :plus_minus: Whether or not to consider plus minus (Y or N)
        :pace_adjust: Whether or not to pace adjust stats (Y or N)
        :rank: Whether or not to consider rank (Y or N)
        :league_id: ID for the league to look in (Default is 00)
        :season: Season given to look up
        :season_type: Season type to consider (Regular / Playoffs)
        :po_round: Playoff round
        :outcome: Filter out by wins or losses
        :location: Filter out by home or away
        :month: Specify month to filter by
        :season_segment: Filter by pre/post all star break
        :date_from: Filter out games before a specific date
        :date_to: Filter out games after a specific date
        :opponent_team_id: Opponent team ID to look up
        :vs_conference: Filter by conference
        :vs_division: Filter by division
        :game_segment: Filter by half / overtime
        :period: Filter by quarter / specific overtime
        :shot_clock_range: Filter statistics by range in shot clock
        :last_n_games: Filter by number of games specified in N

    Attributes:
        :json: Contains the full json dump to play around with
    """
    _endpoint = 'playerdashptshotdefend'


class PlayerShotLogTracking(_PlayerDashboard):
    """
    Contains a log for every shot for a given season for a given player

    Args:
        :player_id: ID of the player to look up
        :team_id: ID of the team to look up
        :measure_type: Specifies type of measure to use (Base, Advanced, etc.)
        :per_mode: Mode to measure statistics (Totals, PerGame, Per36, etc.)
        :plus_minus: Whether or not to consider plus minus (Y or N)
        :pace_adjust: Whether or not to pace adjust stats (Y or N)
        :rank: Whether or not to consider rank (Y or N)
        :league_id: ID for the league to look in (Default is 00)
        :season: Season given to look up
        :season_type: Season type to consider (Regular / Playoffs)
        :po_round: Playoff round
        :outcome: Filter out by wins or losses
        :location: Filter out by home or away
        :month: Specify month to filter by
        :season_segment: Filter by pre/post all star break
        :date_from: Filter out games before a specific date
        :date_to: Filter out games after a specific date
        :opponent_team_id: Opponent team ID to look up
        :vs_conference: Filter by conference
        :vs_division: Filter by division
        :game_segment: Filter by half / overtime
        :period: Filter by quarter / specific overtime
        :shot_clock_range: Filter statistics by range in shot clock
        :last_n_games: Filter by number of games specified in N

    Attributes:
        :json: Contains the full json dump to play around with
    """
    _endpoint = 'playerdashptshotlog'


class PlayerReboundLogTracking(_PlayerDashboard):
    """
    Contains a log for every rebound for a given season for a given player

    Args:
        :player_id: ID of the player to look up
        :team_id: ID of the team to look up
        :measure_type: Specifies type of measure to use (Base, Advanced, etc.)
        :per_mode: Mode to measure statistics (Totals, PerGame, Per36, etc.)
        :plus_minus: Whether or not to consider plus minus (Y or N)
        :pace_adjust: Whether or not to pace adjust stats (Y or N)
        :rank: Whether or not to consider rank (Y or N)
        :league_id: ID for the league to look in (Default is 00)
        :season: Season given to look up
        :season_type: Season type to consider (Regular / Playoffs)
        :po_round: Playoff round
        :outcome: Filter out by wins or losses
        :location: Filter out by home or away
        :month: Specify month to filter by
        :season_segment: Filter by pre/post all star break
        :date_from: Filter out games before a specific date
        :date_to: Filter out games after a specific date
        :opponent_team_id: Opponent team ID to look up
        :vs_conference: Filter by conference
        :vs_division: Filter by division
        :game_segment: Filter by half / overtime
        :period: Filter by quarter / specific overtime
        :shot_clock_range: Filter statistics by range in shot clock
        :last_n_games: Filter by number of games specified in N

    Attributes:
        :json: Contains the full json dump to play around with
    """
    _endpoint = 'playerdashptreboundlogs'


class PlayerVsPlayer:
    """
    Contains general stats that pertain to players going against other players

    Args:
        :player_id: ID of the player to look up
        :vs_player_id: ID of the vs player to look up
        :team_id: ID of the team to look up
        :measure_type: Specifies type of measure to use (Base, Advanced, etc.)
        :per_mode: Mode to measure statistics (Totals, PerGame, Per36, etc.)
        :plus_minus: Whether or not to consider plus minus (Y or N)
        :pace_adjust: Whether or not to pace adjust stats (Y or N)
        :rank: Whether or not to consider rank (Y or N)
        :league_id: ID for the league to look in (Default is 00)
        :season: Season given to look up
        :season_type: Season type to consider (Regular / Playoffs)
        :po_round: Playoff round
        :outcome: Filter out by wins or losses
        :location: Filter out by home or away
        :month: Specify month to filter by
        :season_segment: Filter by pre/post all star break
        :date_from: Filter out games before a specific date
        :date_to: Filter out games after a specific date
        :opponent_team_id: Opponent team ID to look up
        :vs_conference: Filter by conference
        :vs_division: Filter by division
        :game_segment: Filter by half / overtime
        :period: Filter by quarter / specific overtime
        :shot_clock_range: Filter statistics by range in shot clock
        :last_n_games: Filter by number of games specified in N

    Attributes:
        json: Contains the full json dump to play around with
    """
    _endpoint = 'playervsplayer'

    def __init__(self,
                 player_id,
                 vs_player_id,
                 team_id=0,
                 measure_type=MeasureType.Default,
                 per_mode=PerMode.Default,
                 plus_minus=PlusMinus.Default,
                 pace_adjust=PaceAdjust.Default,
                 rank=PaceAdjust.Default,
                 league_id=League.Default,
                 season=CURRENT_SEASON,
                 season_type=SeasonType.Default,
                 po_round=PlayoffRound.Default,
                 outcome=Outcome.Default,
                 location=Location.Default,
                 month=Month.Default,
                 season_segment=SeasonSegment.Default,
                 date_from=DateFrom.Default,
                 date_to=DateTo.Default,
                 opponent_team_id=OpponentTeamID.Default,
                 vs_conference=VsConference.Default,
                 vs_division=VsDivision.Default,
                 game_segment=GameSegment.Default,
                 period=Period.Default,
                 shot_clock_range=ShotClockRange.Default,
                 last_n_games=LastNGames.Default):
        self.json = _get_json(endpoint=self._endpoint,
                              params={'PlayerID': player_id,
                                      'VsPlayerID': vs_player_id,
                                      'TeamID': team_id,
                                      'MeasureType': measure_type,
                                      'PerMode': per_mode,
                                      'PlusMinus': plus_minus,
                                      'PaceAdjust': pace_adjust,
                                      'Rank': rank,
                                      'LeagueID': league_id,
                                      'Season': season,
                                      'SeasonType': season_type,
                                      'PORound': po_round,
                                      'Outcome': outcome,
                                      'Location': location,
                                      'Month': month,
                                      'SeasonSegment': season_segment,
                                      'DateFrom': date_from,
                                      'DateTo': date_to,
                                      'OpponentTeamID': opponent_team_id,
                                      'VsConference': vs_conference,
                                      'VsDivision': vs_division,
                                      'GameSegment': game_segment,
                                      'Period': period,
                                      'ShotClockRange': shot_clock_range,
                                      'LastNGames': last_n_games})

    def overall(self):
        return _api_scrape(self.json, 0)

    def on_off_court(self):
        return _api_scrape(self.json, 1)

    def shot_distance_overall(self):
        return _api_scrape(self.json, 2)

    def shot_distance_on_court(self):
        return _api_scrape(self.json, 3)

    def shot_distance_off_court(self):
        return _api_scrape(self.json, 4)

    def shot_area_overall(self):
        return _api_scrape(self.json, 5)

    def shot_area_on_court(self):
        return _api_scrape(self.json, 6)

    def shot_area_off_court(self):
        return _api_scrape(self.json, 7)

    def player_info(self):
        return _api_scrape(self.json, 8)

    def vs_player_info(self):
        return _api_scrape(self.json, 9)

ImportError: cannot import name HAS_PANDAS

In [ ]:
from nba_py import _api_scrape, _get_json
from nba_py.constants import *


class TeamList:
    _endpoint = 'commonteamyears'

    def __init__(self,
                 league_id=League.NBA):
        self.json = _get_json(endpoint=self._endpoint,
                              params={'LeagueID': league_id})

    def info(self):
        return _api_scrape(self.json, 0)


class TeamSummary:
    _endpoint = 'teaminfocommon'

    def __init__(self,
                 team_id,
                 season=CURRENT_SEASON,
                 league_id=League.NBA,
                 season_type=SeasonType.Default):
        self.json = _get_json(endpoint=self._endpoint,
                              params={'TeamID': team_id,
                                      'Season': season,
                                      'LeagueID': league_id,
                                      'SeasonType': season_type})

    def info(self):
        return _api_scrape(self.json, 0)

    def season_ranks(self):
        return _api_scrape(self.json, 1)


class TeamCommonRoster:
    _endpoint = 'commonteamroster'

    def __init__(self,
                 team_id,
                 season=CURRENT_SEASON):
        self.json = _get_json(endpoint=self._endpoint,
                              params={'TeamID': team_id,
                                      'Season': season})

    def roster(self):
        return _api_scrape(self.json, 0)

    def coaches(self):
        return _api_scrape(self.json, 1)


class _TeamDashboard:
    _endpoint = ''

    def __init__(self,
                 team_id,
                 measure_type=MeasureType.Default,
                 per_mode=PerMode.Default,
                 plus_minus=PlusMinus.Default,
                 pace_adjust=PaceAdjust.Default,
                 rank=Rank.Default,
                 league_id=League.Default,
                 season=CURRENT_SEASON,
                 season_type=SeasonType.Default,
                 po_round=PlayoffRound.Default,
                 outcome=Outcome.Default,
                 location=Location.Default,
                 month=Month.Default,
                 season_segment=SeasonSegment.Default,
                 date_from=DateFrom.Default,
                 date_to=DateTo.Default,
                 opponent_team_id=OpponentTeamID.Default,
                 vs_conference=VsConference.Default,
                 vs_division=VsDivision.Default,
                 game_segment=GameSegment.Default,
                 period=Period.Default,
                 shot_clock_range=ShotClockRange.Default,
                 last_n_games=LastNGames.Default):
        self.json = _get_json(endpoint=self._endpoint,
                              params={'TeamID': team_id,
                                      'MeasureType': measure_type,
                                      'PerMode': per_mode,
                                      'PlusMinus': plus_minus,
                                      'PaceAdjust': pace_adjust,
                                      'Rank': rank,
                                      'LeagueID': league_id,
                                      'Season': season,
                                      'SeasonType': season_type,
                                      'PORound': po_round,
                                      'Outcome': outcome,
                                      'Location': location,
                                      'Month': month,
                                      'SeasonSegment': season_segment,
                                      'DateFrom': date_from,
                                      'DateTo': date_to,
                                      'OpponentTeamID': opponent_team_id,
                                      'VsConference': vs_conference,
                                      'VsDivision': vs_division,
                                      'GameSegment': game_segment,
                                      'Period': period,
                                      'ShotClockRange': shot_clock_range,
                                      'LastNGames': last_n_games})

    def overall(self):
        return _api_scrape(self.json, 0)


class TeamGeneralSplits(_TeamDashboard):
    _endpoint = 'teamdashboardbygeneralsplits'

    def location(self):
        return _api_scrape(self.json, 1)

    def wins_losses(self):
        return _api_scrape(self.json, 2)

    def monthly(self):
        return _api_scrape(self.json, 3)

    def pre_post_all_star(self):
        return _api_scrape(self.json, 4)

    def days_rest(self):
        return _api_scrape(self.json, 5)


class TeamOpponentSplits(_TeamDashboard):
    _endpoint = 'teamdashboardbyopponent'

    def by_conference(self):
        return _api_scrape(self.json, 1)

    def by_division(self):
        return _api_scrape(self.json, 2)

    def by_opponent(self):
        return _api_scrape(self.json, 2)


class TeamLastNGamesSplits(_TeamDashboard):
    _endpoint = 'teamdashboardbylastngames'

    def last5(self):
        return _api_scrape(self.json, 1)

    def last10(self):
        return _api_scrape(self.json, 2)

    def last15(self):
        return _api_scrape(self.json, 3)

    def last20(self):
        return _api_scrape(self.json, 4)

    def gamenumber(self):
        return _api_scrape(self.json, 5)


class TeamInGameSplits(_TeamDashboard):
    _endpoint = 'teamdashboardbygamesplits'

    def by_half(self):
        return _api_scrape(self.json, 1)

    def by_period(self):
        return _api_scrape(self.json, 2)

    def by_score_margin(self):
        return _api_scrape(self.json, 3)

    def by_actual_margin(self):
        return _api_scrape(self.json, 4)


class TeamClutchSplits(_TeamDashboard):
    """
    This is a weird endpoint, to be honest.
    It's got a lot of cool little stats and there are two extra
    fields in the json that I have no idea what they do.

    If you know please tell me.
        * Last30Sec3Point2TeamDashboard
        * Last10Sec3Point2TeamDashboard
    """
    _endpoint = 'teamdashboardbyclutch'

    def last5min_deficit_5point(self):
        """
        Results in last 5 minutes <= 5 points
        """
        return _api_scrape(self.json, 1)

    def last3min_deficit_5point(self):
        """
        Results in last 5 minutes <= 5 points
        """
        return _api_scrape(self.json, 2)

    def last1min_deficit_5point(self):
        """
        Results in last 5 minutes <= 5 points
        """
        return _api_scrape(self.json, 3)

    def last30sec_deficit_3point(self):
        """
        Results in last 5 minutes <= 5 points
        """
        return _api_scrape(self.json, 4)

    def last10sec_deficit_3point(self):
        """
        Results in last 5 minutes <= 5 points
        """
        return _api_scrape(self.json, 5)

    def last5min_plusminus_5point(self):
        """
        Last 5 minutes +/= 5 points
        """
        return _api_scrape(self.json, 6)

    def last3min_plusminus_5point(self):
        """
        Last 3 minutes +/= 5 points
        """
        return _api_scrape(self.json, 7)

    def last1min_plusminus_5point(self):
        """
        Last 1 minutes +/= 5 points
        """
        return _api_scrape(self.json, 8)

    def last30sec_plusminus_5point(self):
        """
        Last 30 seconds +/= 3 points
        """
        return _api_scrape(self.json, 9)


class TeamShootingSplits(_TeamDashboard):
    _endpoint = 'teamdashboardbyshootingsplits'

    def shot_5ft(self):
        return _api_scrape(self.json, 1)

    def shot_8ft(self):
        return _api_scrape(self.json, 2)

    def shot_areas(self):
        return _api_scrape(self.json, 3)

    def assisted_shots(self):
        return _api_scrape(self.json, 4)

    def shot_type_summary(self):
        return _api_scrape(self.json, 5)

    def shot_type_detail(self):
        return _api_scrape(self.json, 6)

    def assissted_by(self):
        return _api_scrape(self.json, 7)


class TeamPerformanceSplits(_TeamDashboard):
    _endpoint = 'teamdashboardbyteamperformance'

    def score_differential(self):
        return _api_scrape(self.json, 1)

    def points_scored(self):
        return _api_scrape(self.json, 2)

    def points_against(self):
        return _api_scrape(self.json, 3)


class TeamYearOverYearSplits(_TeamDashboard):
    _endpoint = 'teamdashboardbyyearoveryear'

    def by_year(self):
        return _api_scrape(self.json, 1)


class TeamLineups:
    _endpoint = 'teamdashlineups'

    def __init__(self,
                 team_id,
                 game_id='',
                 group_quantity=GroupQuantity.Default,
                 season=CURRENT_SEASON,
                 season_type=SeasonType.Default,
                 measure_type=MeasureType.Default,
                 per_mode=PerMode.Default,
                 plus_minus=PlusMinus.Default,
                 pace_adjust=PaceAdjust.Default,
                 rank=Rank.Default,
                 outcome=Outcome.Default,
                 location=Location.Default,
                 month=Month.Default,
                 season_segment=SeasonSegment.Default,
                 date_from=DateFrom.Default,
                 date_to=DateTo.Default,
                 opponent_team_id=OpponentTeamID.Default,
                 vs_conference=VsConference.Default,
                 vs_division=VsDivision.Default,
                 game_segment=GameSegment.Default,
                 period=Period.Default,
                 last_n_games=LastNGames.Default):
        self.json = _get_json(endpoint=self._endpoint,
                              params={'GroupQuantity': group_quantity,
                                      'GameID': game_id,
                                      'TeamID': team_id,
                                      'Season': season,
                                      'SeasonType': season_type,
                                      'MeasureType': measure_type,
                                      'PerMode': per_mode,
                                      'PlusMinus': plus_minus,
                                      'PaceAdjust': pace_adjust,
                                      'Rank': rank,
                                      'Outcome': outcome,
                                      'Location': location,
                                      'Month': month,
                                      'SeasonSegment': season_segment,
                                      'DateFrom': date_from,
                                      'DateTo': date_to,
                                      'OpponentTeamID': opponent_team_id,
                                      'VsConference': vs_conference,
                                      'VsDivision': vs_division,
                                      'GameSegment': game_segment,
                                      'Period': period,
                                      'LastNGames': last_n_games})

    def overall(self):
        return _api_scrape(self.json, 0)

    def lineups(self):
        return _api_scrape(self.json, 1)


class TeamPlayers(_TeamDashboard):
    _endpoint = 'teamplayerdashboard'

    def season_totals(self):
        return _api_scrape(self.json, 1)


class TeamPlayerOnOffDetail(_TeamDashboard):
    _endpoint = 'teamplayeronoffdetails'

    def on_court(self):
        return _api_scrape(self.json, 1)

    def off_court(self):
        return _api_scrape(self.json, 2)


class TeamPlayerOnOffSummary(_TeamDashboard):
    _endpoint = 'teamplayeronoffsummary'

    def on_court(self):
        return _api_scrape(self.json, 1)

    def off_court(self):
        return _api_scrape(self.json, 2)


class TeamGameLogs:
    _endpoint = 'teamgamelog'

    def __init__(self,
                 team_id,
                 season=CURRENT_SEASON,
                 season_type=SeasonType.Default):
        self.json = _get_json(endpoint=self._endpoint,
                              params={'TeamID': team_id,
                                      'Season': season,
                                      'SeasonType': season_type})

    def info(self):
        return _api_scrape(self.json, 0)


class TeamSeasons:
    _endpoint = 'teamyearbyyearstats'

    def __init__(self,
                 team_id,
                 league_id=League.NBA,
                 season_type=SeasonType.Default,
                 per_mode=PerMode.Default):
        self.json = _get_json(endpoint=self._endpoint,
                              params={'TeamID': team_id,
                                      'LeagueID': league_id,
                                      'SeasonType': season_type,
                                      'PerMode': per_mode})

    def info(self):
        return _api_scrape(self.json, 0)


class TeamShotTracking(_TeamDashboard):
    _endpoint = 'teamdashptshots'

    def shot_clock_shooting(self):
        return _api_scrape(self.json, 1)

    def dribble_shooting(self):
        return _api_scrape(self.json, 2)

    def closest_defender_shooting(self):
        return _api_scrape(self.json, 3)

    def closest_defender_shooting_long(self):
        return _api_scrape(self.json, 4)

    def touch_time_shooting(self):
        return _api_scrape(self.json, 5)


class TeamReboundTracking(_TeamDashboard):
    _endpoint = 'teamdashptreb'

    def shot_type_rebounding(self):
        return _api_scrape(self.json, 1)

    def contested_rebounding(self):
        return _api_scrape(self.json, 2)

    def shot_distance_rebounding(self):
        return _api_scrape(self.json, 3)

    def rebound_distance_rebounding(self):
        return _api_scrape(self.json, 4)


class TeamPassTracking(_TeamDashboard):
    _endpoint = 'teamdashptpass'

    def passes_made(self):
        return _api_scrape(self.json, 0)

    def passes_recieved(self):
        return _api_scrape(self.json, 1)


class TeamVsPlayer:
    _endpoint = 'teamvsplayer'

    def __init__(self,
                 team_id,
                 vs_player_id,
                 measure_type=MeasureType.Default,
                 per_mode=PerMode.Default,
                 plus_minus=PlusMinus.Default,
                 pace_adjust=PaceAdjust.Default,
                 rank=Rank.Default,
                 league_id=League.Default,
                 season=CURRENT_SEASON,
                 season_type=SeasonType.Default,
                 po_round=PlayoffRound.Default,
                 outcome=Outcome.Default,
                 location=Location.Default,
                 month=Month.Default,
                 season_segment=SeasonSegment.Default,
                 date_from=DateFrom.Default,
                 date_to=DateTo.Default,
                 opponent_team_id=OpponentTeamID.Default,
                 vs_conference=VsConference.Default,
                 vs_division=VsDivision.Default,
                 game_segment=GameSegment.Default,
                 period=Period.Default,
                 shot_clock_range=ShotClockRange.Default,
                 last_n_games=LastNGames.Default):
        self.json = _get_json(endpoint=self._endpoint,
                              params={'TeamID': team_id,
                                      'VsPlayerID': vs_player_id,
                                      'MeasureType': measure_type,
                                      'PerMode': per_mode,
                                      'PlusMinus': plus_minus,
                                      'PaceAdjust': pace_adjust,
                                      'Rank': rank,
                                      'LeagueID': league_id,
                                      'Season': season,
                                      'SeasonType': season_type,
                                      'PORound': po_round,
                                      'Outcome': outcome,
                                      'Location': location,
                                      'Month': month,
                                      'SeasonSegment': season_segment,
                                      'DateFrom': date_from,
                                      'DateTo': date_to,
                                      'OpponentTeamID': opponent_team_id,
                                      'VsConference': vs_conference,
                                      'VsDivision': vs_division,
                                      'GameSegment': game_segment,
                                      'Period': period,
                                      'ShotClockRange': shot_clock_range,
                                      'LastNGames': last_n_games})

    def overall(self):
        return _api_scrape(self.json, 0)

    def vs_player_overall(self):
        return _api_scrape(self.json, 1)

    def on_off_court(self):
        return _api_scrape(self.json, 2)

    def shot_distance_overall(self):
        return _api_scrape(self.json, 3)

    def shot_distance_on_court(self):
        return _api_scrape(self.json, 4)

    def shot_distance_off_court(self):
        return _api_scrape(self.json, 5)

    def shot_area_overall(self):
        return _api_scrape(self.json, 6)

    def shot_area_on_court(self):
        return _api_scrape(self.json, 7)

    def shot_area_off_court(self):
        return _api_scrape(self.json, 8)

In [ ]:
from nba_py import _api_scrape, _get_json
from nba_py.constants import *


class PlayerNotFoundException(Exception):
    pass


def get_player(first_name,
               last_name,
               season=CURRENT_SEASON,
               only_current=0,
               just_id=True):
    """
    Calls our PlayerList class to get a full list of players and then returns
    just an id if specified or the full row of player information

    Args:
        :first_name: First name of the player
        :last_name: Last name of the player
        :only_current: Only wants the current list of players
        :just_id: Only wants the id of the player

    Returns:
        Either the ID or full row of information of the player inputted

    Raises:
        :PlayerNotFoundException::
    """
    name = '{}, {}'.format(last_name, first_name).lower()
    pl = PlayerList(season=season, only_current=only_current).info()
    hdr = 'DISPLAY_LAST_COMMA_FIRST'
    if HAS_PANDAS:
        item = pl[pl.DISPLAY_LAST_COMMA_FIRST.str.lower() == name]
    else:
        item = next(plyr for plyr in pl if str(plyr[hdr]).lower() == name)
    if len(item) == 0:
        raise PlayerNotFoundException
    elif just_id:
        return item['PERSON_ID']
    else:
        return item


class PlayerList:
    """
    Contains a list of all players for a season, if specified, and will only
    contain current players if specified as well

    Args:
        :league_id: ID for the league to look in (Default is 00)
        :season: Season given to look up
        :only_current: Restrict lookup to only current players

    Attributes:
        :json: Contains the full json dump to play around with
    """
    _endpoint = 'commonallplayers'

    def __init__(self,
                 league_id=League.NBA,
                 season=CURRENT_SEASON,
                 only_current=1):
        self.json = _get_json(endpoint=self._endpoint,
                              params={'LeagueID': league_id,
                                      'Season': season,
                                      'IsOnlyCurrentSeason': only_current})

    def info(self):
        return _api_scrape(self.json, 0)


class PlayerSummary:
    """
    Contains common player information like headline stats, weight, etc.

    Args:
        :player_id: ID of the player to look up

    Attributes:
        :json: Contains the full json dump to play around with
    """
    _endpoint = 'commonplayerinfo'

    def __init__(self,
                 player_id):
        self.json = _get_json(endpoint=self._endpoint,
                              params={'PlayerID': player_id})

    def info(self):
        return _api_scrape(self.json, 0)

    def headline_stats(self):
        return _api_scrape(self.json, 1)


class _PlayerDashboard:
    """
    Has all the basic arguments for all of the Player Dashboard type objects

    Args:
        :player_id: ID of the player to look up
        :team_id: ID of the team to look up
        :measure_type: Specifies type of measure to use (Base, Advanced, etc.)
        :per_mode: Mode to measure statistics (Totals, PerGame, Per36, etc.)
        :plus_minus: Whether or not to consider plus minus (Y or N)
        :pace_adjust: Whether or not to pace adjust stats (Y or N)
        :rank: Whether or not to consider rank (Y or N)
        :league_id: ID for the league to look in (Default is 00)
        :season: Season given to look up
        :season_type: Season type to consider (Regular / Playoffs)
        :po_round: Playoff round
        :outcome: Filter out by wins or losses
        :location: Filter out by home or away
        :month: Specify month to filter by
        :season_segment: Filter by pre/post all star break
        :date_from: Filter out games before a specific date
        :date_to: Filter out games after a specific date
        :opponent_team_id: Opponent team ID to look up
        :vs_conference: Filter by conference
        :vs_division: Filter by division
        :game_segment: Filter by half / overtime
        :period: Filter by quarter / specific overtime
        :shot_clock_range: Filter statistics by range in shot clock
        :last_n_games: Filter by number of games specified in N

    Attributes:
        :json: Contains the full json dump to play around with
    """
    _endpoint = ''

    def __init__(self,
                 player_id,
                 team_id=0,
                 measure_type=MeasureType.Default,
                 per_mode=PerMode.Default,
                 plus_minus=PlusMinus.Default,
                 pace_adjust=PaceAdjust.Default,
                 rank=PaceAdjust.Default,
                 league_id=League.Default,
                 season=CURRENT_SEASON,
                 season_type=SeasonType.Default,
                 po_round=PlayoffRound.Default,
                 outcome=Outcome.Default,
                 location=Location.Default,
                 month=Month.Default,
                 season_segment=SeasonSegment.Default,
                 date_from=DateFrom.Default,
                 date_to=DateTo.Default,
                 opponent_team_id=OpponentTeamID.Default,
                 vs_conference=VsConference.Default,
                 vs_division=VsDivision.Default,
                 game_segment=GameSegment.Default,
                 period=Period.Default,
                 shot_clock_range=ShotClockRange.Default,
                 last_n_games=LastNGames.Default):
        self.json = _get_json(endpoint=self._endpoint,
                              params={'PlayerID': player_id,
                                      'TeamID': team_id,
                                      'MeasureType': measure_type,
                                      'PerMode': per_mode,
                                      'PlusMinus': plus_minus,
                                      'PaceAdjust': pace_adjust,
                                      'Rank': rank,
                                      'LeagueID': league_id,
                                      'Season': season,
                                      'SeasonType': season_type,
                                      'PORound': po_round,
                                      'Outcome': outcome,
                                      'Location': location,
                                      'Month': month,
                                      'SeasonSegment': season_segment,
                                      'DateFrom': date_from,
                                      'DateTo': date_to,
                                      'OpponentTeamID': opponent_team_id,
                                      'VsConference': vs_conference,
                                      'VsDivision': vs_division,
                                      'GameSegment': game_segment,
                                      'Period': period,
                                      'ShotClockRange': shot_clock_range,
                                      'LastNGames': last_n_games})

    def overall(self):
        return _api_scrape(self.json, 0)


class PlayerGeneralSplits(_PlayerDashboard):
    """

    UNFORTUNATELY MARKED AS FORBIDDEN BY THE NBA

    Contains stats pertaining to location, wins and losses, pre/post all star
    break, starting position, and numbers of days rest

    Args:
        :player_id: ID of the player to look up
        :team_id: ID of the team to look up
        :measure_type: Specifies type of measure to use (Base, Advanced, etc.)
        :per_mode: Mode to measure statistics (Totals, PerGame, Per36, etc.)
        :plus_minus: Whether or not to consider plus minus (Y or N)
        :pace_adjust: Whether or not to pace adjust stats (Y or N)
        :rank: Whether or not to consider rank (Y or N)
        :league_id: ID for the league to look in (Default is 00)
        :season: Season given to look up
        :season_type: Season type to consider (Regular / Playoffs)
        :po_round: Playoff round
        :outcome: Filter out by wins or losses
        :location: Filter out by home or away
        :month: Specify month to filter by
        :season_segment: Filter by pre/post all star break
        :date_from: Filter out games before a specific date
        :date_to: Filter out games after a specific date
        :opponent_team_id: Opponent team ID to look up
        :vs_conference: Filter by conference
        :vs_division: Filter by division
        :game_segment: Filter by half / overtime
        :period: Filter by quarter / specific overtime
        :shot_clock_range: Filter statistics by range in shot clock
        :last_n_games: Filter by number of games specified in N

    Attributes:
        :json: Contains the full json dump to play around with
    """
    _endpoint = 'playerdashboardbygeneralsplits'

    def location(self):
        return _api_scrape(self.json, 1)

    def win_losses(self):
        return _api_scrape(self.json, 2)

    def pre_post_all_star(self):
        return _api_scrape(self.json, 3)

    def starting_position(self):
        return _api_scrape(self.json, 4)

    def days_rest(self):
        return _api_scrape(self.json, 5)


class PlayerOpponentSplits(_PlayerDashboard):
    """

    UNFORTUNATELY MARKED AS FORBIDDEN BY THE NBA

    Contains stats pertaining to player stats vs certain opponents by division,
    conference, and by specific team opponent

    Args:
        :player_id: ID of the player to look up
        :team_id: ID of the team to look up
        :measure_type: Specifies type of measure to use (Base, Advanced, etc.)
        :per_mode: Mode to measure statistics (Totals, PerGame, Per36, etc.)
        :plus_minus: Whether or not to consider plus minus (Y or N)
        :pace_adjust: Whether or not to pace adjust stats (Y or N)
        :rank: Whether or not to consider rank (Y or N)
        :league_id: ID for the league to look in (Default is 00)
        :season: Season given to look up
        :season_type: Season type to consider (Regular / Playoffs)
        :po_round: Playoff round
        :outcome: Filter out by wins or losses
        :location: Filter out by home or away
        :month: Specify month to filter by
        :season_segment: Filter by pre/post all star break
        :date_from: Filter out games before a specific date
        :date_to: Filter out games after a specific date
        :opponent_team_id: Opponent team ID to look up
        :vs_conference: Filter by conference
        :vs_division: Filter by division
        :game_segment: Filter by half / overtime
        :period: Filter by quarter / specific overtime
        :shot_clock_range: Filter statistics by range in shot clock
        :last_n_games: Filter by number of games specified in N

    Attributes:
        :json: Contains the full json dump to play around with
    """
    _endpoint = 'playerdashboardbyopponent'

    def by_conference(self):
        return _api_scrape(self.json, 1)

    def by_division(self):
        return _api_scrape(self.json, 2)

    def by_opponent(self):
        return _api_scrape(self.json, 3)


class PlayerLastNGamesSplits(_PlayerDashboard):
    """
    Contains players stats per last 5, 10, 15, and 20 games, or specified
    number of games.

    Args:
        :player_id: ID of the player to look up
        :team_id: ID of the team to look up
        :measure_type: Specifies type of measure to use (Base, Advanced, etc.)
        :per_mode: Mode to measure statistics (Totals, PerGame, Per36, etc.)
        :plus_minus: Whether or not to consider plus minus (Y or N)
        :pace_adjust: Whether or not to pace adjust stats (Y or N)
        :rank: Whether or not to consider rank (Y or N)
        :league_id: ID for the league to look in (Default is 00)
        :season: Season given to look up
        :season_type: Season type to consider (Regular / Playoffs)
        :po_round: Playoff round
        :outcome: Filter out by wins or losses
        :location: Filter out by home or away
        :month: Specify month to filter by
        :season_segment: Filter by pre/post all star break
        :date_from: Filter out games before a specific date
        :date_to: Filter out games after a specific date
        :opponent_team_id: Opponent team ID to look up
        :vs_conference: Filter by conference
        :vs_division: Filter by division
        :game_segment: Filter by half / overtime
        :period: Filter by quarter / specific overtime
        :shot_clock_range: Filter statistics by range in shot clock
        :last_n_games: Filter by number of games specified in N

    Attributes:
        :json: Contains the full json dump to play around with
    """
    _endpoint = 'playerdashboardbylastngames'

    def last5(self):
        return _api_scrape(self.json, 1)

    def last10(self):
        return _api_scrape(self.json, 2)

    def last15(self):
        return _api_scrape(self.json, 3)

    def last20(self):
        return _api_scrape(self.json, 4)

    def gamenumber(self):
        return _api_scrape(self.json, 5)


class PlayerInGameSplits(_PlayerDashboard):
    """
    Contains player stats by half, by quarter, by score margin, and by actual
    margins.

    Args:
        :player_id: ID of the player to look up
        :team_id: ID of the team to look up
        :measure_type: Specifies type of measure to use (Base, Advanced, etc.)
        :per_mode: Mode to measure statistics (Totals, PerGame, Per36, etc.)
        :plus_minus: Whether or not to consider plus minus (Y or N)
        :pace_adjust: Whether or not to pace adjust stats (Y or N)
        :rank: Whether or not to consider rank (Y or N)
        :league_id: ID for the league to look in (Default is 00)
        :season: Season given to look up
        :season_type: Season type to consider (Regular / Playoffs)
        :po_round: Playoff round
        :outcome: Filter out by wins or losses
        :location: Filter out by home or away
        :month: Specify month to filter by
        :season_segment: Filter by pre/post all star break
        :date_from: Filter out games before a specific date
        :date_to: Filter out games after a specific date
        :opponent_team_id: Opponent team ID to look up
        :vs_conference: Filter by conference
        :vs_division: Filter by division
        :game_segment: Filter by half / overtime
        :period: Filter by quarter / specific overtime
        :shot_clock_range: Filter statistics by range in shot clock
        :last_n_games: Filter by number of games specified in N

    Attributes:
        :json: Contains the full json dump to play around with
    """
    _endpoint = 'playerdashboardbygamesplits'

    def by_half(self):
        return _api_scrape(self.json, 1)

    def by_period(self):
        return _api_scrape(self.json, 2)

    def by_score_margin(self):
        return _api_scrape(self.json, 3)

    def by_actual_margin(self):
        return _api_scrape(self.json, 4)


class PlayerClutchSplits(_PlayerDashboard):
    """
    Contains a lot of methods for last n minutes with a deficit of x points

    Args:
        :player_id: ID of the player to look up
        :team_id: ID of the team to look up
        :measure_type: Specifies type of measure to use (Base, Advanced, etc.)
        :per_mode: Mode to measure statistics (Totals, PerGame, Per36, etc.)
        :plus_minus: Whether or not to consider plus minus (Y or N)
        :pace_adjust: Whether or not to pace adjust stats (Y or N)
        :rank: Whether or not to consider rank (Y or N)
        :league_id: ID for the league to look in (Default is 00)
        :season: Season given to look up
        :season_type: Season type to consider (Regular / Playoffs)
        :po_round: Playoff round
        :outcome: Filter out by wins or losses
        :location: Filter out by home or away
        :month: Specify month to filter by
        :season_segment: Filter by pre/post all star break
        :date_from: Filter out games before a specific date
        :date_to: Filter out games after a specific date
        :opponent_team_id: Opponent team ID to look up
        :vs_conference: Filter by conference
        :vs_division: Filter by division
        :game_segment: Filter by half / overtime
        :period: Filter by quarter / specific overtime
        :shot_clock_range: Filter statistics by range in shot clock
        :last_n_games: Filter by number of games specified in N

    Attributes:
        :json: Contains the full json dump to play around with
    """
    _endpoint = 'playerdashboardbyclutch'

    def last5min_deficit_5point(self):
        """
        Results in last 5 minutes <= 5 points
        """
        return _api_scrape(self.json, 1)

    def last3min_deficit_5point(self):
        """
        Results in last 5 minutes <= 5 points
        """
        return _api_scrape(self.json, 2)

    def last1min_deficit_5point(self):
        """
        Results in last 5 minutes <= 5 points
        """
        return _api_scrape(self.json, 3)

    def last30sec_deficit_3point(self):
        """
        Results in last 5 minutes <= 5 points
        """
        return _api_scrape(self.json, 4)

    def last10sec_deficit_3point(self):
        """
        Results in last 5 minutes <= 5 points
        """
        return _api_scrape(self.json, 5)

    def last5min_plusminus_5point(self):
        """
        Last 5 minutes +/= 5 points
        """
        return _api_scrape(self.json, 6)

    def last3min_plusminus_5point(self):
        """
        Last 3 minutes +/= 5 points
        """
        return _api_scrape(self.json, 7)

    def last1min_plusminus_5point(self):
        """
        Last 1 minutes +/= 5 points
        """
        return _api_scrape(self.json, 8)

    def last30sec_plusminus_5point(self):
        """
        Last 30 seconds +/= 3 points
        """
        return _api_scrape(self.json, 9)


class PlayerShootingSplits(_PlayerDashboard):
    """

    UNFORTUNATELY MARKED AS FORBIDDEN BY THE NBA

    Shooting stats based on distance, area, assisted to, shot types, and
    assisted by.

    Args:
        :player_id: ID of the player to look up
        :team_id: ID of the team to look up
        :measure_type: Specifies type of measure to use (Base, Advanced, etc.)
        :per_mode: Mode to measure statistics (Totals, PerGame, Per36, etc.)
        :plus_minus: Whether or not to consider plus minus (Y or N)
        :pace_adjust: Whether or not to pace adjust stats (Y or N)
        :rank: Whether or not to consider rank (Y or N)
        :league_id: ID for the league to look in (Default is 00)
        :season: Season given to look up
        :season_type: Season type to consider (Regular / Playoffs)
        :po_round: Playoff round
        :outcome: Filter out by wins or losses
        :location: Filter out by home or away
        :month: Specify month to filter by
        :season_segment: Filter by pre/post all star break
        :date_from: Filter out games before a specific date
        :date_to: Filter out games after a specific date
        :opponent_team_id: Opponent team ID to look up
        :vs_conference: Filter by conference
        :vs_division: Filter by division
        :game_segment: Filter by half / overtime
        :period: Filter by quarter / specific overtime
        :shot_clock_range: Filter statistics by range in shot clock
        :last_n_games: Filter by number of games specified in N

    Attributes:
        :json: Contains the full json dump to play around with
    """
    _endpoint = 'playerdashboardbyshootingsplits'

    def shot_5ft(self):
        return _api_scrape(self.json, 1)

    def shot_8ft(self):
        return _api_scrape(self.json, 2)

    def shot_areas(self):
        return _api_scrape(self.json, 3)

    def assisted_shots(self):
        return _api_scrape(self.json, 4)

    def shot_types_summary(self):
        return _api_scrape(self.json, 5)

    def shot_types_detail(self):
        return _api_scrape(self.json, 6)

    def assissted_by(self):
        return _api_scrape(self.json, 7)


class PlayerPerformanceSplits(_PlayerDashboard):
    """
    Player stats by different performance metrics such as score differntial,
    points scored, and points scored against

    Args:
        :player_id: ID of the player to look up
        :team_id: ID of the team to look up
        :measure_type: Specifies type of measure to use (Base, Advanced, etc.)
        :per_mode: Mode to measure statistics (Totals, PerGame, Per36, etc.)
        :plus_minus: Whether or not to consider plus minus (Y or N)
        :pace_adjust: Whether or not to pace adjust stats (Y or N)
        :rank: Whether or not to consider rank (Y or N)
        :league_id: ID for the league to look in (Default is 00)
        :season: Season given to look up
        :season_type: Season type to consider (Regular / Playoffs)
        :po_round: Playoff round
        :outcome: Filter out by wins or losses
        :location: Filter out by home or away
        :month: Specify month to filter by
        :season_segment: Filter by pre/post all star break
        :date_from: Filter out games before a specific date
        :date_to: Filter out games after a specific date
        :opponent_team_id: Opponent team ID to look up
        :vs_conference: Filter by conference
        :vs_division: Filter by division
        :game_segment: Filter by half / overtime
        :period: Filter by quarter / specific overtime
        :shot_clock_range: Filter statistics by range in shot clock
        :last_n_games: Filter by number of games specified in N

    Attributes:
        :json: Contains the full json dump to play around with
    """
    _endpoint = 'playerdashboardbyteamperformance'

    def score_differential(self):
        return _api_scrape(self.json, 1)

    def points_scored(self):
        return _api_scrape(self.json, 2)

    def points_against(self):
        return _api_scrape(self.json, 3)


class PlayerYearOverYearSplits(_PlayerDashboard):
    """
    Displays player stats over the given season and over all seasons in the
    given league

    Args:
        :player_id: ID of the player to look up
        :team_id: ID of the team to look up
        :measure_type: Specifies type of measure to use (Base, Advanced, etc.)
        :per_mode: Mode to measure statistics (Totals, PerGame, Per36, etc.)
        :plus_minus: Whether or not to consider plus minus (Y or N)
        :pace_adjust: Whether or not to pace adjust stats (Y or N)
        :rank: Whether or not to consider rank (Y or N)
        :league_id: ID for the league to look in (Default is 00)
        :season: Season given to look up
        :season_type: Season type to consider (Regular / Playoffs)
        :po_round: Playoff round
        :outcome: Filter out by wins or losses
        :location: Filter out by home or away
        :month: Specify month to filter by
        :season_segment: Filter by pre/post all star break
        :date_from: Filter out games before a specific date
        :date_to: Filter out games after a specific date
        :opponent_team_id: Opponent team ID to look up
        :vs_conference: Filter by conference
        :vs_division: Filter by division
        :game_segment: Filter by half / overtime
        :period: Filter by quarter / specific overtime
        :shot_clock_range: Filter statistics by range in shot clock
        :last_n_games: Filter by number of games specified in N

    Attributes:
        :json: Contains the full json dump to play around with
    """
    _endpoint = 'playerdashboardbyyearoveryear'

    def by_year(self):
        return _api_scrape(self.json, 1)


class PlayerCareer:
    """
    Contains stats based on several parameters such as career regular season
    totals, post season career totals, all star season careers totals, college
    season career totals, etc.

    Args:
        :player_id: Player ID to look up
        :per_mode: Mode to measure statistics (Totals, PerGame, Per36, etc.)
        :league_id: ID for the league to look in (Default is 00)

    Attributes:
        :json: Contains the full json dump to play around with
    """
    _endpoint = 'playercareerstats'

    def __init__(self,
                 player_id,
                 per_mode=PerMode.PerGame,
                 league_id=League.NBA):
        self.json = _get_json(endpoint=self._endpoint,
                              params={'PlayerID': player_id,
                                      'LeagueID': league_id,
                                      'PerMode': per_mode})

    def regular_season_totals(self):
        return _api_scrape(self.json, 0)

    def regular_season_career_totals(self):
        return _api_scrape(self.json, 1)

    def post_season_totals(self):
        return _api_scrape(self.json, 2)

    def post_season_career_totals(self):
        return _api_scrape(self.json, 3)

    def all_star_season_totals(self):
        return _api_scrape(self.json, 4)

    def career_all_star_season_totals(self):
        return _api_scrape(self.json, 5)

    def college_season_totals(self):
        return _api_scrape(self.json, 6)

    def college_season_career_totals(self):
        return _api_scrape(self.json, 7)

    def regular_season_rankings(self):
        return _api_scrape(self.json, 8)

    def post_season_rankings(self):
        return _api_scrape(self.json, 9)


class PlayerProfile(PlayerCareer):
    """
    Contains a more in depth version of player career stats with season highs,
    career highs, and when the player's next game is

    Args:
        :player_id: Player ID to look up
        :per_mode: Mode to measure statistics (Totals, PerGame, Per36, etc.)
        :league_id: ID for the league to look in (Default is 00)

    Attributes:
        :json: Contains the full json dump to play around with
    """
    _endpoint = 'playerprofilev2'

    def season_highs(self):
        return _api_scrape(self.json, 10)

    def career_highs(self):
        return _api_scrape(self.json, 11)

    def next_game(self):
        return _api_scrape(self.json, 12)


class PlayerGameLogs:
    """
    Contains a full log of all the games for a player for a given season

    Args:
        :player_id: ID of the player to look up
        :league_id: ID for the league to look in (Default is 00)
        :season: Season given to look up
        :season_type: Season type to consider (Regular / Playoffs)

    Attributes:
        :json: Contains the full json dump to play around with
    """
    _endpoint = 'playergamelog'

    def __init__(self,
                 player_id,
                 league_id=League.NBA,
                 season=CURRENT_SEASON,
                 season_type=SeasonType.Regular):
        self.json = _get_json(endpoint=self._endpoint,
                              params={'PlayerID': player_id,
                                      'LeagueID': league_id,
                                      'Season': season,
                                      'SeasonType': season_type})

    def info(self):
        return _api_scrape(self.json, 0)


class PlayerShotTracking(_PlayerDashboard):
    """

    UNFORTUNATELY MARKED AS FORBIDDEN BY THE NBA

    Tracking data for shooting for a given player

    Args:
        :player_id: ID of the player to look up
        :team_id: ID of the team to look up
        :measure_type: Specifies type of measure to use (Base, Advanced, etc.)
        :per_mode: Mode to measure statistics (Totals, PerGame, Per36, etc.)
        :plus_minus: Whether or not to consider plus minus (Y or N)
        :pace_adjust: Whether or not to pace adjust stats (Y or N)
        :rank: Whether or not to consider rank (Y or N)
        :league_id: ID for the league to look in (Default is 00)
        :season: Season given to look up
        :season_type: Season type to consider (Regular / Playoffs)
        :po_round: Playoff round
        :outcome: Filter out by wins or losses
        :location: Filter out by home or away
        :month: Specify month to filter by
        :season_segment: Filter by pre/post all star break
        :date_from: Filter out games before a specific date
        :date_to: Filter out games after a specific date
        :opponent_team_id: Opponent team ID to look up
        :vs_conference: Filter by conference
        :vs_division: Filter by division
        :game_segment: Filter by half / overtime
        :period: Filter by quarter / specific overtime
        :shot_clock_range: Filter statistics by range in shot clock
        :last_n_games: Filter by number of games specified in N

    Attributes:
        :json: Contains the full json dump to play around with
    """
    _endpoint = 'playerdashptshots'

    def general_shooting(self):
        return _api_scrape(self.json, 1)

    def shot_clock_shooting(self):
        return _api_scrape(self.json, 2)

    def dribble_shooting(self):
        return _api_scrape(self.json, 3)

    def closest_defender_shooting(self):
        return _api_scrape(self.json, 4)

    def closest_defender_shooting_long(self):
        return _api_scrape(self.json, 5)

    def touch_time_shooting(self):
        return _api_scrape(self.json, 6)


class PlayerReboundTracking(_PlayerDashboard):
    """

    UNFORTUNATELY MARKED AS FORBIDDEN BY THE NBA

    Tracking data for rebounding for a given player

    Args:
        :player_id: ID of the player to look up
        :team_id: ID of the team to look up
        :measure_type: Specifies type of measure to use (Base, Advanced, etc.)
        :per_mode: Mode to measure statistics (Totals, PerGame, Per36, etc.)
        :plus_minus: Whether or not to consider plus minus (Y or N)
        :pace_adjust: Whether or not to pace adjust stats (Y or N)
        :rank: Whether or not to consider rank (Y or N)
        :league_id: ID for the league to look in (Default is 00)
        :season: Season given to look up
        :season_type: Season type to consider (Regular / Playoffs)
        :po_round: Playoff round
        :outcome: Filter out by wins or losses
        :location: Filter out by home or away
        :month: Specify month to filter by
        :season_segment: Filter by pre/post all star break
        :date_from: Filter out games before a specific date
        :date_to: Filter out games after a specific date
        :opponent_team_id: Opponent team ID to look up
        :vs_conference: Filter by conference
        :vs_division: Filter by division
        :game_segment: Filter by half / overtime
        :period: Filter by quarter / specific overtime
        :shot_clock_range: Filter statistics by range in shot clock
        :last_n_games: Filter by number of games specified in N

    Attributes:
        :json: Contains the full json dump to play around with
    """
    _endpoint = 'playerdashptreb'

    def shot_type_rebounding(self):
        return _api_scrape(self.json, 1)

    def num_contested_rebounding(self):
        return _api_scrape(self.json, 2)

    def shot_distance_rebounding(self):
        return _api_scrape(self.json, 3)

    def rebound_distance_rebounding(self):
        return _api_scrape(self.json, 4)


class PlayerPassTracking(_PlayerDashboard):
    """
    Tracking data for passing for a given player

    Args:
        :player_id: ID of the player to look up
        :team_id: ID of the team to look up
        :measure_type: Specifies type of measure to use (Base, Advanced, etc.)
        :per_mode: Mode to measure statistics (Totals, PerGame, Per36, etc.)
        :plus_minus: Whether or not to consider plus minus (Y or N)
        :pace_adjust: Whether or not to pace adjust stats (Y or N)
        :rank: Whether or not to consider rank (Y or N)
        :league_id: ID for the league to look in (Default is 00)
        :season: Season given to look up
        :season_type: Season type to consider (Regular / Playoffs)
        :po_round: Playoff round
        :outcome: Filter out by wins or losses
        :location: Filter out by home or away
        :month: Specify month to filter by
        :season_segment: Filter by pre/post all star break
        :date_from: Filter out games before a specific date
        :date_to: Filter out games after a specific date
        :opponent_team_id: Opponent team ID to look up
        :vs_conference: Filter by conference
        :vs_division: Filter by division
        :game_segment: Filter by half / overtime
        :period: Filter by quarter / specific overtime
        :shot_clock_range: Filter statistics by range in shot clock
        :last_n_games: Filter by number of games specified in N

    Attributes:
        :json: Contains the full json dump to play around with
    """
    _endpoint = 'playerdashptpass'

    def passes_made(self):
        return _api_scrape(self.json, 0)

    def passes_recieved(self):
        return _api_scrape(self.json, 1)


class PlayerDefenseTracking(_PlayerDashboard):
    """
    Tracking data for defense for a given player

    Args:
        :player_id: ID of the player to look up
        :team_id: ID of the team to look up
        :measure_type: Specifies type of measure to use (Base, Advanced, etc.)
        :per_mode: Mode to measure statistics (Totals, PerGame, Per36, etc.)
        :plus_minus: Whether or not to consider plus minus (Y or N)
        :pace_adjust: Whether or not to pace adjust stats (Y or N)
        :rank: Whether or not to consider rank (Y or N)
        :league_id: ID for the league to look in (Default is 00)
        :season: Season given to look up
        :season_type: Season type to consider (Regular / Playoffs)
        :po_round: Playoff round
        :outcome: Filter out by wins or losses
        :location: Filter out by home or away
        :month: Specify month to filter by
        :season_segment: Filter by pre/post all star break
        :date_from: Filter out games before a specific date
        :date_to: Filter out games after a specific date
        :opponent_team_id: Opponent team ID to look up
        :vs_conference: Filter by conference
        :vs_division: Filter by division
        :game_segment: Filter by half / overtime
        :period: Filter by quarter / specific overtime
        :shot_clock_range: Filter statistics by range in shot clock
        :last_n_games: Filter by number of games specified in N

    Attributes:
        :json: Contains the full json dump to play around with
    """
    _endpoint = 'playerdashptshotdefend'


class PlayerShotLogTracking(_PlayerDashboard):
    """
    Contains a log for every shot for a given season for a given player

    Args:
        :player_id: ID of the player to look up
        :team_id: ID of the team to look up
        :measure_type: Specifies type of measure to use (Base, Advanced, etc.)
        :per_mode: Mode to measure statistics (Totals, PerGame, Per36, etc.)
        :plus_minus: Whether or not to consider plus minus (Y or N)
        :pace_adjust: Whether or not to pace adjust stats (Y or N)
        :rank: Whether or not to consider rank (Y or N)
        :league_id: ID for the league to look in (Default is 00)
        :season: Season given to look up
        :season_type: Season type to consider (Regular / Playoffs)
        :po_round: Playoff round
        :outcome: Filter out by wins or losses
        :location: Filter out by home or away
        :month: Specify month to filter by
        :season_segment: Filter by pre/post all star break
        :date_from: Filter out games before a specific date
        :date_to: Filter out games after a specific date
        :opponent_team_id: Opponent team ID to look up
        :vs_conference: Filter by conference
        :vs_division: Filter by division
        :game_segment: Filter by half / overtime
        :period: Filter by quarter / specific overtime
        :shot_clock_range: Filter statistics by range in shot clock
        :last_n_games: Filter by number of games specified in N

    Attributes:
        :json: Contains the full json dump to play around with
    """
    _endpoint = 'playerdashptshotlog'


class PlayerReboundLogTracking(_PlayerDashboard):
    """
    Contains a log for every rebound for a given season for a given player

    Args:
        :player_id: ID of the player to look up
        :team_id: ID of the team to look up
        :measure_type: Specifies type of measure to use (Base, Advanced, etc.)
        :per_mode: Mode to measure statistics (Totals, PerGame, Per36, etc.)
        :plus_minus: Whether or not to consider plus minus (Y or N)
        :pace_adjust: Whether or not to pace adjust stats (Y or N)
        :rank: Whether or not to consider rank (Y or N)
        :league_id: ID for the league to look in (Default is 00)
        :season: Season given to look up
        :season_type: Season type to consider (Regular / Playoffs)
        :po_round: Playoff round
        :outcome: Filter out by wins or losses
        :location: Filter out by home or away
        :month: Specify month to filter by
        :season_segment: Filter by pre/post all star break
        :date_from: Filter out games before a specific date
        :date_to: Filter out games after a specific date
        :opponent_team_id: Opponent team ID to look up
        :vs_conference: Filter by conference
        :vs_division: Filter by division
        :game_segment: Filter by half / overtime
        :period: Filter by quarter / specific overtime
        :shot_clock_range: Filter statistics by range in shot clock
        :last_n_games: Filter by number of games specified in N

    Attributes:
        :json: Contains the full json dump to play around with
    """
    _endpoint = 'playerdashptreboundlogs'


class PlayerVsPlayer:
    """
    Contains general stats that pertain to players going against other players

    Args:
        :player_id: ID of the player to look up
        :vs_player_id: ID of the vs player to look up
        :team_id: ID of the team to look up
        :measure_type: Specifies type of measure to use (Base, Advanced, etc.)
        :per_mode: Mode to measure statistics (Totals, PerGame, Per36, etc.)
        :plus_minus: Whether or not to consider plus minus (Y or N)
        :pace_adjust: Whether or not to pace adjust stats (Y or N)
        :rank: Whether or not to consider rank (Y or N)
        :league_id: ID for the league to look in (Default is 00)
        :season: Season given to look up
        :season_type: Season type to consider (Regular / Playoffs)
        :po_round: Playoff round
        :outcome: Filter out by wins or losses
        :location: Filter out by home or away
        :month: Specify month to filter by
        :season_segment: Filter by pre/post all star break
        :date_from: Filter out games before a specific date
        :date_to: Filter out games after a specific date
        :opponent_team_id: Opponent team ID to look up
        :vs_conference: Filter by conference
        :vs_division: Filter by division
        :game_segment: Filter by half / overtime
        :period: Filter by quarter / specific overtime
        :shot_clock_range: Filter statistics by range in shot clock
        :last_n_games: Filter by number of games specified in N

    Attributes:
        json: Contains the full json dump to play around with
    """
    _endpoint = 'playervsplayer'

    def __init__(self,
                 player_id,
                 vs_player_id,
                 team_id=0,
                 measure_type=MeasureType.Default,
                 per_mode=PerMode.Default,
                 plus_minus=PlusMinus.Default,
                 pace_adjust=PaceAdjust.Default,
                 rank=PaceAdjust.Default,
                 league_id=League.Default,
                 season=CURRENT_SEASON,
                 season_type=SeasonType.Default,
                 po_round=PlayoffRound.Default,
                 outcome=Outcome.Default,
                 location=Location.Default,
                 month=Month.Default,
                 season_segment=SeasonSegment.Default,
                 date_from=DateFrom.Default,
                 date_to=DateTo.Default,
                 opponent_team_id=OpponentTeamID.Default,
                 vs_conference=VsConference.Default,
                 vs_division=VsDivision.Default,
                 game_segment=GameSegment.Default,
                 period=Period.Default,
                 shot_clock_range=ShotClockRange.Default,
                 last_n_games=LastNGames.Default):
        self.json = _get_json(endpoint=self._endpoint,
                              params={'PlayerID': player_id,
                                      'VsPlayerID': vs_player_id,
                                      'TeamID': team_id,
                                      'MeasureType': measure_type,
                                      'PerMode': per_mode,
                                      'PlusMinus': plus_minus,
                                      'PaceAdjust': pace_adjust,
                                      'Rank': rank,
                                      'LeagueID': league_id,
                                      'Season': season,
                                      'SeasonType': season_type,
                                      'PORound': po_round,
                                      'Outcome': outcome,
                                      'Location': location,
                                      'Month': month,
                                      'SeasonSegment': season_segment,
                                      'DateFrom': date_from,
                                      'DateTo': date_to,
                                      'OpponentTeamID': opponent_team_id,
                                      'VsConference': vs_conference,
                                      'VsDivision': vs_division,
                                      'GameSegment': game_segment,
                                      'Period': period,
                                      'ShotClockRange': shot_clock_range,
                                      'LastNGames': last_n_games})

    def overall(self):
        return _api_scrape(self.json, 0)

    def on_off_court(self):
        return _api_scrape(self.json, 1)

    def shot_distance_overall(self):
        return _api_scrape(self.json, 2)

    def shot_distance_on_court(self):
        return _api_scrape(self.json, 3)

    def shot_distance_off_court(self):
        return _api_scrape(self.json, 4)

    def shot_area_overall(self):
        return _api_scrape(self.json, 5)

    def shot_area_on_court(self):
        return _api_scrape(self.json, 6)

    def shot_area_off_court(self):
        return _api_scrape(self.json, 7)

    def player_info(self):
        return _api_scrape(self.json, 8)

    def vs_player_info(self):
        return _api_scrape(self.json, 9)

In [ ]:
stats.nba.com/stats/_PlayerDashboard/?{params}